In [5]:
import pandas as pd
import numpy as np

from sklearn.model_selection import *
from sklearn.metrics import *

from sklearn.neighbors import *
from sklearn.ensemble import *
from sklearn.tree import *
from sklearn.linear_model import *
from sklearn.svm import *
from sklearn.decomposition import *

import xgboost as xgb
import lightgbm as lgb

# import tensorflow as tf

import os
import re
import ast

In [6]:
directory_dataframes = '/Users/nitanshjain/Documents/Projects/Shopper_Intent_Prediction/shopper-intent-prediction/long_trajectory/subsamples/'
directory_features = '/Users/nitanshjain/Documents/Projects/Shopper_Intent_Prediction/shopper-intent-prediction/long_trajectory/features/'

def get_sample_df(directory=directory_dataframes):
    filename_list = []
    list_dataframes = []
    for filename in os.listdir(directory):
        print(filename)
        filename_list.append(filename)
        f = os.path.join(directory, filename)
        if os.path.isfile(f):
            list_dataframes.append(pd.read_csv(f))
            
    return list_dataframes, filename_list

def get_features(regex_str, directory=directory_features):
    regex = re.compile('/Users/nitanshjain/Documents/Projects/Shopper_Intent_Prediction/shopper-intent-prediction/long_trajectory/features/{}'.format(regex_str))
    
    for filename in os.listdir(directory):
        f = os.path.join(directory, filename)
        if regex.match(f):
            file1 = open(f,"r+")
            feat_list = file1.read().splitlines()
            
            #txt file converts everything to string, so we need to convert it back to list
            for i in range(len(feat_list)):
                #adding ; to be used a separator for list
                if i<len(feat_list):
                    new_val = feat_list[i].replace('y','y;').replace(') ','); ').replace('4 ', '4; ').replace('5 ', '5; ')
                    feat_list[i] = new_val
                
    for val in feat_list:
        #separating the string into a list of features
        new_val = val.split('; ')
        feat_list[feat_list.index(val)] = new_val
        
    return feat_list

list_sample_dataframes, filename_sample_list = get_sample_df(directory_dataframes)

subsample_8.csv


subsample_9.csv
subsample_7.csv
subsample_6.csv
subsample_10.csv
subsample_4.csv
subsample_5.csv
subsample_1.csv
subsample_2.csv
subsample_3.csv


In [7]:
def model_train_predict(model, regex_str, dataframes=list_sample_dataframes, params=None):
    
    feat_list = get_features(regex_str)
    
    accuracy_list = []
    f1_score_list = []
    auc_list = []
    best_params_list = []
    
    for sample, feat in zip(dataframes, feat_list):
        feat[len(feat)-1] = feat[len(feat)-1].replace('y;', 'y')
        x = sample[feat]
        x = x.rename(columns = lambda a:re.sub('[^A-Za-z0-9_]+', '', a))
        
        y = sample['conversion_class']
        x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42, stratify=y)
        clf = GridSearchCV(estimator=model, param_grid=params, cv=5, n_jobs=-1)
        clf.fit(x_train, y_train)
        y_pred = clf.predict(x_test)
        
        
        # model.fit(x_train, y_train)
        # y_pred = model.predict(x_test)
        accuracy_list.append(accuracy_score(y_test, y_pred))
        f1_score_list.append(f1_score(y_test, y_pred))
        auc_list.append(roc_auc_score(y_test, y_pred))
        best_params_list.append(clf.best_params_)

    print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)
    
    print('Average Accuracy', np.mean(accuracy_list))
    print('Average F1 Score', np.mean(f1_score_list))
    print('Average AUC', np.mean(auc_list)) 
    
    print('Max Accuracy', max(accuracy_list))
    print('Max F1 Score', max(f1_score_list))
    print('Max AUC', max(auc_list))  
    
    print(accuracy_list)
    print(auc_list)
    best_accuracy_index = accuracy_list.index(max(accuracy_list))
    best_f1_score_index = f1_score_list.index(max(f1_score_list))
    best_auc_index = auc_list.index(max(auc_list))
    
    print('Best Sample Index based on Max Accuracy', best_accuracy_index)
    print('Best Sample Index based on Max F1 Score', best_f1_score_index)
    print('Best Sample Index based on Max AUC', best_auc_index)
    
    print('Best Features based on Max Accuracy', feat_list[best_accuracy_index])
    print('Best Features based on Max F1 Score', feat_list[best_f1_score_index])
    print('Best Features based on Max AUC', feat_list[best_auc_index]) 
    print('Best Params based on Max Accuracy', best_params_list[best_accuracy_index])
    
    return accuracy_list, f1_score_list, auc_list, best_params_list  


# Mutual Information

## 10 Percentile

In [8]:
lr = LogisticRegression()
params = {'C': [1, 10, 100, 1000], 'max_iter': [1000, 2000, 5000, 10000]}
accuracy_list_lr_10_mi, f1_score_list_lr_10_mi, auc_list_lr_10_mi, param_list_lr_10_mi = model_train_predict(lr, 'mi_feat_list_10', params=params)
print("\n================================================================\n")
rfc = RandomForestClassifier()
params = {'n_estimators': [10, 50, 100], 'max_depth': [3, 5, 10]}
accuracy_list_rfc_10_mi, f1_score_list_rfc_10_mi, auc_list_rfc_10_mi, param_list_rfc_10_mi = model_train_predict(rfc, 'mi_feat_list_10', params=params)
print("\n================================================================\n")
svc = SVC()
params = {'C': [1, 10, 100], 'kernel': ['linear', 'poly', 'rbf', 'sigmoid'], 'gamma': ['auto', 'scale'],}
accuracy_list_svm_10_mi, f1_score_list_svm_10_mi, auc_list_svm_10_mi, param_list_svm_10_mi = model_train_predict(svc, 'mi_feat_list_10', params=params)
print("\n================================================================\n")
xgbc = xgb.XGBClassifier()
params = {'n_estimators': [10, 50, 100, 200, 500], 'max_depth': [3, 5, 10, 20, 50, 100]}
accuracy_list_xgb_10_mi, f1_score_list_xgb_10_mi, auc_list_xgb_10_mi, param_list_xgb_10_mi = model_train_predict(xgbc, 'mi_feat_list_10', params=params)
print("\n================================================================\n")
lgbc = lgb.LGBMClassifier()
params = {'learning_rate': [0.1, 0.05, 0.01], 'n_estimators': [10, 50, 100], 'max_depth': [3, 5, 10]}
accuracy_list_lgb_10_mi, f1_score_list_lgb_10_mi, auc_list_lgb_10_mi, param_list_lgb_10_mi = model_train_predict(lgbc, 'mi_feat_list_10', params=params)

(12052, 16) (12052,) (3014, 16) (3014,)
Average Accuracy 0.8471466489714663
Average F1 Score 0.8436537371510958
Average AUC 0.8471466489714665
Max Accuracy 0.8526874585268746
Max F1 Score 0.8480492813141685
Max AUC 0.8526874585268747
[0.8437292634372926, 0.8526874585268746, 0.8513603185136032, 0.8443928334439283, 0.843065693430657, 0.8510285335102853, 0.845056403450564, 0.8487060384870604, 0.8463835434638355, 0.845056403450564]
[0.8437292634372926, 0.8526874585268747, 0.8513603185136033, 0.8443928334439283, 0.8430656934306568, 0.8510285335102852, 0.8450564034505641, 0.8487060384870604, 0.8463835434638354, 0.845056403450564]
Best Sample Index based on Max Accuracy 1
Best Sample Index based on Max F1 Score 1
Best Sample Index based on Max AUC 1
Best Features based on Max Accuracy ['unigram_entropy', 'bigram_entropy', 'trigram_entropy', '(2,)', '(3,)', '(4,)', '(1, 2)', '(2, 1)', '(2, 3)', '(3, 1)', '(1, 4)', '(4, 1)', '(1, 2, 1)', '(2, 1, 2)', '(2, 3, 1)', '(3, 1, 1)']
Best Features base

## 20 Percentile

In [9]:
lr = LogisticRegression()
params = {'C': [1, 10, 100, 1000], 'max_iter': [1000, 2000, 5000, 10000]}
accuracy_list_lr_20_mi, f1_score_list_lr_20_mi, auc_list_lr_20_mi, param_list_lr_20_mi = model_train_predict(lr, 'mi_feat_list_20', params=params)
print("\n================================================================\n")
rfc = RandomForestClassifier()
params = {'n_estimators': [10, 50, 100], 'max_depth': [3, 5, 10]}
accuracy_list_rfc_20_mi, f1_score_list_rfc_20_mi, auc_list_rfc_20_mi, param_list_rfc_20_mi = model_train_predict(rfc, 'mi_feat_list_20', params=params)
print("\n================================================================\n")
svc = SVC()
params = {'C': [ 1, 10, 100], 'kernel': ['linear', 'poly', 'rbf', 'sigmoid'], 'gamma': ['auto', 'scale'],}
accuracy_list_svm_20_mi, f1_score_list_svm_20_mi, auc_list_svm_20_mi, param_list_svm_20_mi = model_train_predict(svc, 'mi_feat_list_20', params=params)
print("\n================================================================\n")
xgbc = xgb.XGBClassifier()
params = {'n_estimators': [10, 50, 100], 'max_depth': [3, 5, 10]}
accuracy_list_xgb_20_mi, f1_score_list_xgb_20_mi, auc_list_xgb_20_mi, param_list_xgb_20_mi = model_train_predict(xgbc, 'mi_feat_list_20', params=params)
print("\n================================================================\n")
lgbc = lgb.LGBMClassifier()
params = {'learning_rate': [0.1, 0.05, 0.01], 'n_estimators': [10, 50, 100], 'max_depth': [3, 5, 10]}
accuracy_list_lgb_20_mi, f1_score_list_lgb_20_mi, auc_list_lgb_20_mi, param_list_lgb_20_mi = model_train_predict(lgbc, 'mi_feat_list_20', params=params)

/Users/nitanshjain/.local/share/virtualenvs/shopper-intent-prediction-L5e-nQUd/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/nitanshjain/.local/share/virtualenvs/shopper-intent-prediction-L5e-nQUd/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documen

(12052, 32) (12052,) (3014, 32) (3014,)
Average Accuracy 0.8686463171864631
Average F1 Score 0.8676758947285117
Average AUC 0.8686463171864631
Max Accuracy 0.8762441937624419
Max F1 Score 0.8743684742337487
Max AUC 0.8762441937624419
[0.8682813536828136, 0.8689449236894492, 0.8669542136695422, 0.8626410086264101, 0.8682813536828136, 0.8686131386861314, 0.8639681486396815, 0.8762441937624419, 0.8699402786994028, 0.8725945587259456]
[0.8682813536828136, 0.8689449236894492, 0.8669542136695421, 0.86264100862641, 0.8682813536828136, 0.8686131386861313, 0.8639681486396815, 0.8762441937624419, 0.8699402786994027, 0.8725945587259456]
Best Sample Index based on Max Accuracy 7
Best Sample Index based on Max F1 Score 7
Best Sample Index based on Max AUC 7
Best Features based on Max Accuracy ['unigram_entropy', 'bigram_entropy', 'trigram_entropy', 'pattern_hvg_4_nodes_entropy', 'pattern_hvg_5_node_entropy', '(2,)', '(3,)', '(4,)', '(1, 1)', '(1, 2)', '(2, 1)', '(2, 3)', '(3, 1)', '(1, 4)', '(4, 1)

## 30 Percentile

In [10]:
lr = LogisticRegression()
params = {'C': [1, 10, 100, 1000], 'max_iter': [1000, 2000, 5000, 10000]}
accuracy_list_lr_30_mi, f1_score_list_lr_30_mi, auc_list_lr_30_mi, param_list_lr_30_mi = model_train_predict(lr, 'mi_feat_list_30', params=params)
print("\n================================================================\n")
rfc = RandomForestClassifier()
params = {'n_estimators': [10, 50, 100], 'max_depth': [3, 5, 10]}
accuracy_list_rfc_30_mi, f1_score_list_rfc_30_mi, auc_list_rfc_30_mi, param_list_rfc_30_mi = model_train_predict(rfc, 'mi_feat_list_30', params=params)
print("\n================================================================\n")
svc = SVC()
params = {'C': [ 1, 10, 100], 'kernel': ['linear', 'poly', 'rbf', 'sigmoid'], 'gamma': ['auto', 'scale'],}
accuracy_list_svm_30_mi, f1_score_list_svm_30_mi, auc_list_svm_30_mi, param_list_svm_30_mi = model_train_predict(svc, 'mi_feat_list_30', params=params)
print("\n================================================================\n")
xgbc = xgb.XGBClassifier()
params = {'n_estimators': [10, 50, 100], 'max_depth': [3, 5, 10]}
accuracy_list_xgb_30_mi, f1_score_list_xgb_30_mi, auc_list_xgb_30_mi, param_list_xgb_30_mi = model_train_predict(xgbc, 'mi_feat_list_30', params=params)
print("\n================================================================\n")
lgbc = lgb.LGBMClassifier()
params = {'learning_rate': [0.1, 0.05, 0.01], 'n_estimators': [10, 50, 100], 'max_depth': [3, 5, 10]}
accuracy_list_lgb_30_mi, f1_score_list_lgb_30_mi, auc_list_lgb_30_mi, param_list_lgb_30_mi = model_train_predict(lgbc, 'mi_feat_list_30', params=params)

/Users/nitanshjain/.local/share/virtualenvs/shopper-intent-prediction-L5e-nQUd/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/nitanshjain/.local/share/virtualenvs/shopper-intent-prediction-L5e-nQUd/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documen

(12052, 48) (12052,) (3014, 48) (3014,)
Average Accuracy 0.8717319177173192
Average F1 Score 0.8709522050020613
Average AUC 0.8717319177173192
Max Accuracy 0.8808891838088918
Max F1 Score 0.8790838666217582
Max AUC 0.8808891838088918
[0.8692767086927671, 0.8749170537491705, 0.8725945587259456, 0.8659588586595885, 0.8709356337093563, 0.8689449236894492, 0.8662906436629064, 0.8808891838088918, 0.8749170537491705, 0.8725945587259456]
[0.8692767086927671, 0.8749170537491705, 0.8725945587259456, 0.8659588586595887, 0.8709356337093563, 0.8689449236894492, 0.8662906436629064, 0.8808891838088918, 0.8749170537491705, 0.8725945587259456]
Best Sample Index based on Max Accuracy 7
Best Sample Index based on Max F1 Score 7
Best Sample Index based on Max AUC 7
Best Features based on Max Accuracy ['unigram_entropy', 'bigram_entropy', 'trigram_entropy', 'pattern_hvg_4_nodes_entropy', 'pattern_hvg_5_node_entropy', '(1,)', '(2,)', '(3,)', '(4,)', '(1, 1)', '(1, 2)', '(2, 1)', '(2, 3)', '(3, 1)', '(1, 4)

## 50 Percentile

In [11]:
lr = LogisticRegression()
params = {'C': [1, 10, 100, 1000], 'max_iter': [1000, 2000, 5000, 10000]}
accuracy_list_lr_50_mi, f1_score_list_lr_50_mi, auc_list_lr_50_mi, param_list_lr_50_mi = model_train_predict(lr, 'mi_feat_list_50', params=params)
print("\n================================================================\n")
rfc = RandomForestClassifier()
params = {'n_estimators': [10, 50, 100], 'max_depth': [3, 5, 10]}
accuracy_list_rfc_50_mi, f1_score_list_rfc_50_mi, auc_list_rfc_50_mi, param_list_rfc_50_mi = model_train_predict(rfc, 'mi_feat_list_50', params=params)
print("\n================================================================\n")
svc = SVC()
params = {'C': [ 1, 10, 100], 'kernel': ['linear', 'poly', 'rbf', 'sigmoid'], 'gamma': ['auto', 'scale'],}
accuracy_list_svm_50_mi, f1_score_list_svm_50_mi, auc_list_svm_50_mi, param_list_svm_50_mi = model_train_predict(svc, 'mi_feat_list_50', params=params)
print("\n================================================================\n")
xgbc = xgb.XGBClassifier()
params = {'n_estimators': [10, 50, 100], 'max_depth': [3, 5, 10]}
accuracy_list_xgb_50_mi, f1_score_list_xgb_50_mi, auc_list_xgb_50_mi, param_list_xgb_50_mi = model_train_predict(xgbc, 'mi_feat_list_50', params=params)
print("\n================================================================\n")
lgbc = lgb.LGBMClassifier()
params = {'learning_rate': [0.1, 0.05, 0.01], 'n_estimators': [10, 50, 100], 'max_depth': [3, 5, 10]}
accuracy_list_lgb_50_mi, f1_score_list_lgb_50_mi, auc_list_lgb_50_mi, param_list_lgb_50_mi = model_train_predict(lgbc, 'mi_feat_list_50', params=params)

/Users/nitanshjain/.local/share/virtualenvs/shopper-intent-prediction-L5e-nQUd/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/nitanshjain/.local/share/virtualenvs/shopper-intent-prediction-L5e-nQUd/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documen

(12052, 80) (12052,) (3014, 80) (3014,)
Average Accuracy 0.8774386197743862
Average F1 Score 0.8769596060160725
Average AUC 0.8774386197743862
Max Accuracy 0.8845388188453882
Max F1 Score 0.8828647925033467
Max AUC 0.8845388188453882
[0.8749170537491705, 0.8812209688122097, 0.8802256138022562, 0.8722627737226277, 0.8712674187126742, 0.8735899137358991, 0.8725945587259456, 0.8845388188453882, 0.8838752488387525, 0.8798938287989383]
[0.8749170537491706, 0.8812209688122097, 0.8802256138022561, 0.8722627737226277, 0.8712674187126742, 0.8735899137358991, 0.8725945587259456, 0.8845388188453882, 0.8838752488387523, 0.8798938287989383]
Best Sample Index based on Max Accuracy 7
Best Sample Index based on Max F1 Score 8
Best Sample Index based on Max AUC 7
Best Features based on Max Accuracy ['unigram_entropy', 'bigram_entropy', 'trigram_entropy', 'pattern_hvg_4_nodes_entropy', 'pattern_hvg_5_node_entropy', '(1,)', '(2,)', '(6,)', '(3,)', '(4,)', '(1, 1)', '(1, 2)', '(2, 6)', '(2, 1)', '(2, 2)',

## 75 Percentile

In [12]:
lr = LogisticRegression()
params = {'C': [1, 10, 100, 1000], 'max_iter': [1000, 2000, 5000, 10000]}
accuracy_list_lr_75_mi, f1_score_list_lr_75_mi, auc_list_lr_75_mi, param_list_lr_75_mi = model_train_predict(lr, 'mi_feat_list_75', params=params)
print("\n================================================================\n")
rfc = RandomForestClassifier()
params = {'n_estimators': [10, 50, 100], 'max_depth': [3, 5, 10]}
accuracy_list_rfc_75_mi, f1_score_list_rfc_75_mi, auc_list_rfc_75_mi, param_list_rfc_75_mi = model_train_predict(rfc, 'mi_feat_list_75', params=params)
print("\n================================================================\n")
svc = SVC()
params = {'C': [ 1, 10, 100], 'kernel': ['linear', 'poly', 'rbf', 'sigmoid'], 'gamma': ['auto', 'scale'],}
accuracy_list_svm_75_mi, f1_score_list_svm_75_mi, auc_list_svm_75_mi, param_list_svm_75_mi = model_train_predict(svc, 'mi_feat_list_75', params=params)
print("\n================================================================\n")
xgbc = xgb.XGBClassifier()
params = {'n_estimators': [10, 50, 100], 'max_depth': [3, 5, 10]}
accuracy_list_xgb_75_mi, f1_score_list_xgb_75_mi, auc_list_xgb_75_mi, param_list_xgb_75_mi = model_train_predict(xgbc, 'mi_feat_list_75', params=params)
print("\n================================================================\n")
lgbc = lgb.LGBMClassifier()
params = {'learning_rate': [0.1, 0.05, 0.01], 'n_estimators': [10, 50, 100], 'max_depth': [3, 5, 10]}
accuracy_list_lgb_75_mi, f1_score_list_lgb_75_mi, auc_list_lgb_75_mi, param_list_lgb_75_mi = model_train_predict(lgbc, 'mi_feat_list_75', params=params)

/Users/nitanshjain/.local/share/virtualenvs/shopper-intent-prediction-L5e-nQUd/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/nitanshjain/.local/share/virtualenvs/shopper-intent-prediction-L5e-nQUd/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documen

(12052, 120) (12052,) (3014, 120) (3014,)
Average Accuracy 0.880092899800929
Average F1 Score 0.8798671504091535
Average AUC 0.880092899800929
Max Accuracy 0.8865295288652952
Max F1 Score 0.8850033624747815
Max AUC 0.8865295288652953
[0.8785666887856669, 0.8842070338420703, 0.8808891838088918, 0.8779031187790312, 0.8699402786994028, 0.8792302587923025, 0.8745852687458527, 0.8865295288652952, 0.8848706038487061, 0.8842070338420703]
[0.8785666887856668, 0.8842070338420704, 0.8808891838088919, 0.877903118779031, 0.8699402786994028, 0.8792302587923025, 0.8745852687458527, 0.8865295288652953, 0.8848706038487061, 0.8842070338420703]
Best Sample Index based on Max Accuracy 7
Best Sample Index based on Max F1 Score 7
Best Sample Index based on Max AUC 7
Best Features based on Max Accuracy ['unigram_entropy', 'bigram_entropy', 'trigram_entropy', 'pattern_hvg_4_nodes_entropy', 'pattern_hvg_5_node_entropy', '(1,)', '(2,)', '(6,)', '(3,)', '(4,)', '(1, 1)', '(1, 2)', '(2, 6)', '(6, 1)', '(2, 1)', 

## 90 Percentile

In [13]:
lr = LogisticRegression()
params = {'C': [1, 10, 100, 1000], 'max_iter': [1000, 2000, 5000, 10000]}
accuracy_list_lr_90_mi, f1_score_list_lr_90_mi, auc_list_lr_90_mi, param_list_lr_90_mi = model_train_predict(lr, 'mi_feat_list_90', params=params)
print("\n================================================================\n")
rfc = RandomForestClassifier()
params = {'n_estimators': [10, 50, 100], 'max_depth': [3, 5, 10]}
accuracy_list_rfc_90_mi, f1_score_list_rfc_90_mi, auc_list_rfc_90_mi, param_list_rfc_90_mi = model_train_predict(rfc, 'mi_feat_list_90', params=params)
print("\n================================================================\n")
svc = SVC()
params = {'C': [ 1, 10, 100], 'kernel': ['linear', 'poly', 'rbf', 'sigmoid'], 'gamma': ['auto', 'scale'],}
accuracy_list_svm_90_mi, f1_score_list_svm_90_mi, auc_list_svm_90_mi, param_list_svm_90_mi = model_train_predict(svc, 'mi_feat_list_90', params=params)
print("\n================================================================\n")
xgbc = xgb.XGBClassifier()
params = {'n_estimators': [10, 50, 100], 'max_depth': [3, 5, 10]}
accuracy_list_xgb_90_mi, f1_score_list_xgb_90_mi, auc_list_xgb_90_mi, param_list_xgb_90_mi = model_train_predict(xgbc, 'mi_feat_list_90', params=params)
print("\n================================================================\n")
lgbc = lgb.LGBMClassifier()
params = {'learning_rate': [0.1, 0.05, 0.01], 'n_estimators': [10, 50, 100], 'max_depth': [3, 5, 10]}
accuracy_list_lgb_90_mi, f1_score_list_lgb_90_mi, auc_list_lgb_90_mi, param_list_lgb_90_mi = model_train_predict(lgbc, 'mi_feat_list_90', params=params)

/Users/nitanshjain/.local/share/virtualenvs/shopper-intent-prediction-L5e-nQUd/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/nitanshjain/.local/share/virtualenvs/shopper-intent-prediction-L5e-nQUd/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documen

(12052, 144) (12052,) (3014, 144) (3014,)
Average Accuracy 0.8808228268082281
Average F1 Score 0.8806410495718463
Average AUC 0.8808228268082281
Max Accuracy 0.8888520238885202
Max F1 Score 0.8872433524065971
Max AUC 0.8888520238885202
[0.8788984737889848, 0.8852023888520238, 0.8822163238221632, 0.8769077637690776, 0.8706038487060385, 0.8798938287989383, 0.8755806237558063, 0.8888520238885202, 0.8861977438619775, 0.8838752488387525]
[0.8788984737889847, 0.8852023888520237, 0.8822163238221632, 0.8769077637690774, 0.8706038487060384, 0.8798938287989383, 0.8755806237558063, 0.8888520238885202, 0.8861977438619775, 0.8838752488387525]
Best Sample Index based on Max Accuracy 7
Best Sample Index based on Max F1 Score 7
Best Sample Index based on Max AUC 7
Best Features based on Max Accuracy ['unigram_entropy', 'bigram_entropy', 'trigram_entropy', 'pattern_hvg_4_nodes_entropy', 'pattern_hvg_5_node_entropy', '(1,)', '(2,)', '(6,)', '(3,)', '(4,)', '(1, 1)', '(1, 2)', '(2, 6)', '(6, 1)', '(2, 1)

# mRMR

## 10 Percentile

In [14]:
lr = LogisticRegression()
params = {'C': [1, 10, 100, 1000], 'max_iter': [1000, 2000, 5000, 10000]}
accuracy_list_lr_10_mrmr, f1_score_list_lr_10_mrmr, auc_list_lr_10_mrmr, param_list_lr_10_mrmr = model_train_predict(lr, 'mrmr_feat_list_10', params=params)
print("\n================================================================\n")
rfc = RandomForestClassifier()
params = {'n_estimators': [10, 50, 100], 'max_depth': [3, 5, 10]}
accuracy_list_rfc_10_mrmr, f1_score_list_rfc_10_mrmr, auc_list_rfc_10_mrmr, param_list_rfc_10_mrmr = model_train_predict(rfc, 'mrmr_feat_list_10', params=params)
print("\n================================================================\n")
svc = SVC()
params = {'C': [ 1, 10, 100], 'kernel': ['linear', 'poly', 'rbf', 'sigmoid'], 'gamma': ['auto', 'scale'],}
accuracy_list_svm_10_mrmr, f1_score_list_svm_10_mrmr, auc_list_svm_10_mrmr, param_list_svm_10_mrmr = model_train_predict(svc, 'mrmr_feat_list_10', params=params)
print("\n================================================================\n")
xgbc = xgb.XGBClassifier()
params = {'n_estimators': [10, 50, 100], 'max_depth': [3, 5, 10]}
accuracy_list_xgb_10_mrmr, f1_score_list_xgb_10_mrmr, auc_list_xgb_10_mrmr, param_list_xgb_10_mrmr = model_train_predict(xgbc, 'mrmr_feat_list_10', params=params)
print("\n================================================================\n")
lgbc = lgb.LGBMClassifier()
params = {'learning_rate': [0.1, 0.05, 0.01], 'n_estimators': [10, 50, 100], 'max_depth': [3, 5, 10]}
accuracy_list_lgb_10_mrmr, f1_score_list_lgb_10_mrmr, auc_list_lgb_10_mrmr, param_list_lgb_10_mrmr = model_train_predict(lgbc, 'mrmr_feat_list_10', params=params)

(12052, 14) (12052,) (3014, 14) (3014,)
Average Accuracy 0.8423689449236894
Average F1 Score 0.839930446336593
Average AUC 0.8423689449236894
Max Accuracy 0.8540145985401459
Max F1 Score 0.8502382573179034
Max AUC 0.8540145985401459
[0.8377571333775713, 0.8540145985401459, 0.843065693430657, 0.836098208360982, 0.8410749834107498, 0.843065693430657, 0.836098208360982, 0.8470471134704711, 0.8460517584605176, 0.8394160583941606]
[0.8377571333775713, 0.8540145985401459, 0.8430656934306571, 0.8360982083609821, 0.8410749834107498, 0.8430656934306571, 0.8360982083609821, 0.8470471134704711, 0.8460517584605174, 0.8394160583941606]
Best Sample Index based on Max Accuracy 1
Best Sample Index based on Max F1 Score 1
Best Sample Index based on Max AUC 1
Best Features based on Max Accuracy ['(3, 1)', '(1, 1, 4)', '(3, 3)', '(2, 1, 4)', '(2, 1)', 'pattern_hvg_5_node_entropy', '(3, 1, 1)', 'bigram_entropy', '(1, 2, 1)', '(2, 3)', '(2,)', 'trigram_entropy', '(1, 2)', 'unigram_entropy']
Best Features b

## 20 Percentile

In [15]:
lr = LogisticRegression()
params = {'C': [1, 10, 100, 1000], 'max_iter': [1000, 2000, 5000, 10000]}
accuracy_list_lr_20_mrmr, f1_score_list_lr_20_mrmr, auc_list_lr_20_mrmr, param_list_lr_20_mrmr = model_train_predict(lr, 'mrmr_feat_list_20*', params=params)
print("\n================================================================\n")
rfc = RandomForestClassifier()
params = {'n_estimators': [10, 50, 100], 'max_depth': [3, 5, 10]}
accuracy_list_rfc_20_mrmr, f1_score_list_rfc_20_mrmr, auc_list_rfc_20_mrmr, param_list_rfc_20_mrmr = model_train_predict(rfc, 'mrmr_feat_list_20', params=params)
print("\n================================================================\n")
svc = SVC()
params = {'C': [ 1, 10, 100], 'kernel': ['linear', 'poly', 'rbf', 'sigmoid'], 'gamma': ['auto', 'scale'],}
accuracy_list_svm_20_mrmr, f1_score_list_svm_20_mrmr, auc_list_svm_20_mrmr, param_list_svm_20_mrmr = model_train_predict(svc, 'mrmr_feat_list_20', params=params)
print("\n================================================================\n")
xgbc = xgb.XGBClassifier()
params = {'n_estimators': [10, 50, 100], 'max_depth': [3, 5, 10]}
accuracy_list_xgb_20_mrmr, f1_score_list_xgb_20_mrmr, auc_list_xgb_20_mrmr, param_list_xgb_20_mrmr = model_train_predict(xgbc, 'mrmr_feat_list_20', params=params)
print("\n================================================================\n")
lgbc = lgb.LGBMClassifier()
params = {'learning_rate': [0.1, 0.05, 0.01], 'n_estimators': [10, 50, 100], 'max_depth': [3, 5, 10]}
accuracy_list_lgb_20_mrmr, f1_score_list_lgb_20_mrmr, auc_list_lgb_20_mrmr, param_list_lgb_20_mrmr = model_train_predict(lgbc, 'mrmr_feat_list_20', params=params)

/Users/nitanshjain/.local/share/virtualenvs/shopper-intent-prediction-L5e-nQUd/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/nitanshjain/.local/share/virtualenvs/shopper-intent-prediction-L5e-nQUd/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documen

(12052, 28) (12052,) (3014, 28) (3014,)
Average Accuracy 0.8655938951559389
Average F1 Score 0.8646945809979355
Average AUC 0.865593895155939
Max Accuracy 0.8749170537491705
Max F1 Score 0.8731920618903464
Max AUC 0.8749170537491705
[0.8642999336429993, 0.8742534837425349, 0.8593231585932316, 0.8603185136031851, 0.8576642335766423, 0.8619774386197744, 0.8706038487060385, 0.8749170537491705, 0.8619774386197744, 0.8706038487060385]
[0.8642999336429993, 0.8742534837425348, 0.8593231585932317, 0.8603185136031851, 0.8576642335766423, 0.8619774386197744, 0.8706038487060385, 0.8749170537491705, 0.8619774386197744, 0.8706038487060385]
Best Sample Index based on Max Accuracy 7
Best Sample Index based on Max F1 Score 7
Best Sample Index based on Max AUC 7
Best Features based on Max Accuracy ['(3, 1)', '(1, 1, 4)', '(3, 3)', '(2, 1, 4)', '(2, 1)', 'pattern_hvg_5_node_entropy', '(3, 1, 1)', 'bigram_entropy', '(1, 2, 1)', '(1, 2)', 'trigram_entropy', '(2, 3)', '(2,)', 'unigram_entropy', '(1, 1, 2)'

## 30 Percentile

In [16]:
lr = LogisticRegression()
params = {'C': [1, 10, 100, 1000], 'max_iter': [1000, 2000, 5000, 10000]}
accuracy_list_lr_30_mrmr, f1_score_list_lr_30_mrmr, auc_list_lr_30_mrmr, param_list_lr_30_mrmr = model_train_predict(lr, 'mrmr_feat_list_30*', params=params)
print("\n================================================================\n")
rfc = RandomForestClassifier()
params = {'n_estimators': [10, 50, 100], 'max_depth': [3, 5, 10]}
accuracy_list_rfc_30_mrmr, f1_score_list_rfc_30_mrmr, auc_list_rfc_30_mrmr, param_list_rfc_30_mrmr = model_train_predict(rfc, 'mrmr_feat_list_30', params=params)
print("\n================================================================\n")
svc = SVC()
params = {'C': [ 1, 10, 100], 'kernel': ['linear', 'poly', 'rbf', 'sigmoid'], 'gamma': ['auto', 'scale'],}
accuracy_list_svm_30_mrmr, f1_score_list_svm_30_mrmr, auc_list_svm_30_mrmr, param_list_svm_30_mrmr = model_train_predict(svc, 'mrmr_feat_list_30', params=params)
print("\n================================================================\n")
xgbc = xgb.XGBClassifier()
params = {'n_estimators': [10, 50, 100], 'max_depth': [3, 5, 10]}
accuracy_list_xgb_30_mrmr, f1_score_list_xgb_30_mrmr, auc_list_xgb_30_mrmr, param_list_xgb_30_mrmr = model_train_predict(xgbc, 'mrmr_feat_list_30', params=params)
print("\n================================================================\n")
lgbc = lgb.LGBMClassifier()
params = {'learning_rate': [0.1, 0.05, 0.01], 'n_estimators': [10, 50, 100], 'max_depth': [3, 5, 10]}
accuracy_list_lgb_30_mrmr, f1_score_list_lgb_30_mrmr, auc_list_lgb_30_mrmr, param_list_lgb_30_mrmr = model_train_predict(lgbc, 'mrmr_feat_list_30', params=params)

/Users/nitanshjain/.local/share/virtualenvs/shopper-intent-prediction-L5e-nQUd/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/nitanshjain/.local/share/virtualenvs/shopper-intent-prediction-L5e-nQUd/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documen

(12052, 42) (12052,) (3014, 42) (3014,)
Average Accuracy 0.8738553417385534
Average F1 Score 0.8734080276717698
Average AUC 0.8738553417385534
Max Accuracy 0.8822163238221632
Max F1 Score 0.8809922896413007
Max AUC 0.8822163238221631
[0.8712674187126742, 0.8798938287989383, 0.8755806237558063, 0.8682813536828136, 0.8715992037159921, 0.8692767086927671, 0.8729263437292635, 0.8822163238221632, 0.873921698739217, 0.8735899137358991]
[0.8712674187126742, 0.8798938287989383, 0.8755806237558064, 0.8682813536828136, 0.8715992037159921, 0.8692767086927671, 0.8729263437292635, 0.8822163238221631, 0.8739216987392171, 0.8735899137358992]
Best Sample Index based on Max Accuracy 7
Best Sample Index based on Max F1 Score 7
Best Sample Index based on Max AUC 7
Best Features based on Max Accuracy ['(3, 1)', '(1, 1, 4)', '(3, 3)', '(2, 1, 4)', '(2, 1)', 'pattern_hvg_5_node_entropy', '(3, 1, 1)', 'bigram_entropy', '(1, 2, 1)', '(1, 2)', 'trigram_entropy', '(2, 3)', '(2,)', 'unigram_entropy', '(1, 1, 2)'

## 50 Percentile

In [17]:
lr = LogisticRegression()
params = {'C': [1, 10, 100, 1000], 'max_iter': [1000, 2000, 5000, 10000]}
accuracy_list_lr_50_mrmr, f1_score_list_lr_50_mrmr, auc_list_lr_50_mrmr, param_list_lr_50_mrmr = model_train_predict(lr, 'mrmr_feat_list_50*', params=params)
print("\n================================================================\n")
rfc = RandomForestClassifier()
params = {'n_estimators': [10, 50, 100], 'max_depth': [3, 5, 10]}
accuracy_list_rfc_50_mrmr, f1_score_list_rfc_50_mrmr, auc_list_rfc_50_mrmr, param_list_rfc_50_mrmr = model_train_predict(rfc, 'mrmr_feat_list_50', params=params)
print("\n================================================================\n")
svc = SVC()
params = {'C': [ 1, 10, 100], 'kernel': ['linear', 'poly', 'rbf', 'sigmoid'], 'gamma': ['auto', 'scale'],}
accuracy_list_svm_50_mrmr, f1_score_list_svm_50_mrmr, auc_list_svm_50_mrmr, param_list_svm_50_mrmr = model_train_predict(svc, 'mrmr_feat_list_50', params=params)
print("\n================================================================\n")
xgbc = xgb.XGBClassifier()
params = {'n_estimators': [10, 50, 100], 'max_depth': [3, 5, 10]}
accuracy_list_xgb_50_mrmr, f1_score_list_xgb_50_mrmr, auc_list_xgb_50_mrmr, param_list_xgb_50_mrmr = model_train_predict(xgbc, 'mrmr_feat_list_50', params=params)
print("\n================================================================\n")
lgbc = lgb.LGBMClassifier()
params = {'learning_rate': [0.1, 0.05, 0.01], 'n_estimators': [10, 50, 100], 'max_depth': [3, 5, 10]}
accuracy_list_lgb_50_mrmr, f1_score_list_lgb_50_mrmr, auc_list_lgb_50_mrmr, param_list_lgb_50_mrmr = model_train_predict(lgbc, 'mrmr_feat_list_50', params=params)

/Users/nitanshjain/.local/share/virtualenvs/shopper-intent-prediction-L5e-nQUd/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/nitanshjain/.local/share/virtualenvs/shopper-intent-prediction-L5e-nQUd/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documen

(12052, 69) (12052,) (3014, 69) (3014,)
Average Accuracy 0.877936297279363
Average F1 Score 0.8775394340910234
Average AUC 0.877936297279363
Max Accuracy 0.8861977438619775
Max F1 Score 0.8845506563446651
Max AUC 0.8861977438619775
[0.8759124087591241, 0.8838752488387525, 0.8795620437956204, 0.8729263437292635, 0.8715992037159921, 0.8755806237558063, 0.8712674187126742, 0.8861977438619775, 0.8812209688122097, 0.8812209688122097]
[0.8759124087591241, 0.8838752488387526, 0.8795620437956205, 0.8729263437292635, 0.8715992037159921, 0.8755806237558063, 0.8712674187126742, 0.8861977438619775, 0.8812209688122099, 0.8812209688122097]
Best Sample Index based on Max Accuracy 7
Best Sample Index based on Max F1 Score 7
Best Sample Index based on Max AUC 7
Best Features based on Max Accuracy ['(3, 1)', '(1, 1, 4)', '(3, 3)', '(2, 1, 4)', '(2, 1)', 'pattern_hvg_5_node_entropy', '(3, 1, 1)', 'bigram_entropy', '(1, 2, 1)', '(1, 2)', 'trigram_entropy', '(2, 3)', '(2,)', 'unigram_entropy', '(1, 1, 2)',

## 75 Percentile

In [18]:
lr = LogisticRegression()
params = {'C': [1, 10, 100, 1000], 'max_iter': [1000, 2000, 5000, 10000]}
accuracy_list_lr_75_mrmr, f1_score_list_lr_75_mrmr, auc_list_lr_75_mrmr, param_list_lr_75_mrmr = model_train_predict(lr, 'mrmr_feat_list_75*', params=params)
print("\n================================================================\n")
rfc = RandomForestClassifier()
params = {'n_estimators': [10, 50, 100], 'max_depth': [3, 5, 10]}
accuracy_list_rfc_75_mrmr, f1_score_list_rfc_75_mrmr, auc_list_rfc_75_mrmr, param_list_rfc_75_mrmr = model_train_predict(rfc, 'mrmr_feat_list_75', params=params)
print("\n================================================================\n")
svc = SVC()
params = {'C': [ 1, 10, 100], 'kernel': ['linear', 'poly', 'rbf', 'sigmoid'], 'gamma': ['auto', 'scale'],}
accuracy_list_svm_75_mrmr, f1_score_list_svm_75_mrmr, auc_list_svm_75_mrmr, param_list_svm_75_mrmr = model_train_predict(svc, 'mrmr_feat_list_75', params=params)
print("\n================================================================\n")
xgbc = xgb.XGBClassifier()
params = {'n_estimators': [10, 50, 100], 'max_depth': [3, 5, 10]}
accuracy_list_xgb_75_mrmr, f1_score_list_xgb_75_mrmr, auc_list_xgb_75_mrmr, param_list_xgb_75_mrmr = model_train_predict(xgbc, 'mrmr_feat_list_75', params=params)
print("\n================================================================\n")
lgbc = lgb.LGBMClassifier()
params = {'learning_rate': [0.1, 0.05, 0.01], 'n_estimators': [10, 50, 100], 'max_depth': [3, 5, 10]}
accuracy_list_lgb_75_mrmr, f1_score_list_lgb_75_mrmr, auc_list_lgb_75_mrmr, param_list_lgb_75_mrmr = model_train_predict(lgbc, 'mrmr_feat_list_75', params=params)

/Users/nitanshjain/.local/share/virtualenvs/shopper-intent-prediction-L5e-nQUd/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/nitanshjain/.local/share/virtualenvs/shopper-intent-prediction-L5e-nQUd/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documen

(12052, 104) (12052,) (3014, 104) (3014,)
Average Accuracy 0.879462508294625
Average F1 Score 0.8791996018521552
Average AUC 0.879462508294625
Max Accuracy 0.8868613138686131
Max F1 Score 0.8853781512605041
Max AUC 0.8868613138686132
[0.8815527538155276, 0.8855341738553417, 0.8832116788321168, 0.8729263437292635, 0.8689449236894492, 0.8792302587923025, 0.8735899137358991, 0.8868613138686131, 0.8815527538155276, 0.8812209688122097]
[0.8815527538155276, 0.8855341738553417, 0.8832116788321168, 0.8729263437292635, 0.8689449236894494, 0.8792302587923025, 0.8735899137358991, 0.8868613138686132, 0.8815527538155276, 0.8812209688122097]
Best Sample Index based on Max Accuracy 7
Best Sample Index based on Max F1 Score 7
Best Sample Index based on Max AUC 7
Best Features based on Max Accuracy ['(3, 1)', '(1, 1, 4)', '(3, 3)', '(2, 1, 4)', '(2, 1)', 'pattern_hvg_5_node_entropy', '(3, 1, 1)', 'bigram_entropy', '(1, 2, 1)', '(1, 2)', 'trigram_entropy', '(2, 3)', '(2,)', 'unigram_entropy', '(1, 1, 2)

## 90 Percentile

In [19]:
lr = LogisticRegression()
params = {'C': [1, 10, 100, 1000], 'max_iter': [1000, 2000, 5000, 10000]}
accuracy_list_lr_90_mrmr, f1_score_list_lr_90_mrmr, auc_list_lr_90_mrmr, param_list_lr_90_mrmr = model_train_predict(lr, 'mrmr_feat_list_90*', params=params)
print("\n================================================================\n")
rfc = RandomForestClassifier()
params = {'n_estimators': [10, 50, 100], 'max_depth': [3, 5, 10]}
accuracy_list_rfc_90_mrmr, f1_score_list_rfc_90_mrmr, auc_list_rfc_90_mrmr, param_list_rfc_90_mrmr = model_train_predict(rfc, 'mrmr_feat_list_90', params=params)
print("\n================================================================\n")
svc = SVC()
params = {'C': [ 1, 10, 100], 'kernel': ['linear', 'poly', 'rbf', 'sigmoid'], 'gamma': ['auto', 'scale'],}
accuracy_list_svm_90_mrmr, f1_score_list_svm_90_mrmr, auc_list_svm_90_mrmr, param_list_svm_90_mrmr = model_train_predict(svc, 'mrmr_feat_list_90', params=params)
print("\n================================================================\n")
xgbc = xgb.XGBClassifier()
params = {'n_estimators': [10, 50, 100], 'max_depth': [3, 5, 10]}
accuracy_list_xgb_90_mrmr, f1_score_list_xgb_90_mrmr, auc_list_xgb_90_mrmr, param_list_xgb_90_mrmr = model_train_predict(xgbc, 'mrmr_feat_list_90', params=params)
print("\n================================================================\n")
lgbc = lgb.LGBMClassifier()
params = {'learning_rate': [0.1, 0.05, 0.01], 'n_estimators': [10, 50, 100], 'max_depth': [3, 5, 10]}
accuracy_list_lgb_90_mrmr, f1_score_list_lgb_90_mrmr, auc_list_lgb_90_mrmr, param_list_lgb_90_mrmr = model_train_predict(lgbc, 'mrmr_feat_list_90', params=params)

/Users/nitanshjain/.local/share/virtualenvs/shopper-intent-prediction-L5e-nQUd/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/nitanshjain/.local/share/virtualenvs/shopper-intent-prediction-L5e-nQUd/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documen

(12052, 125) (12052,) (3014, 125) (3014,)
Average Accuracy 0.8809223623092235
Average F1 Score 0.8806490128731606
Average AUC 0.8809223623092235
Max Accuracy 0.8881884538818845
Max F1 Score 0.8864937689457729
Max AUC 0.8881884538818845
[0.8812209688122097, 0.8855341738553417, 0.8832116788321168, 0.8755806237558063, 0.869608493696085, 0.8808891838088918, 0.8792302587923025, 0.8881884538818845, 0.8825481088254811, 0.8832116788321168]
[0.8812209688122097, 0.8855341738553417, 0.8832116788321169, 0.8755806237558063, 0.869608493696085, 0.8808891838088918, 0.8792302587923025, 0.8881884538818845, 0.8825481088254812, 0.8832116788321167]
Best Sample Index based on Max Accuracy 7
Best Sample Index based on Max F1 Score 7
Best Sample Index based on Max AUC 7
Best Features based on Max Accuracy ['(3, 1)', '(1, 1, 4)', '(3, 3)', '(2, 1, 4)', '(2, 1)', 'pattern_hvg_5_node_entropy', '(3, 1, 1)', 'bigram_entropy', '(1, 2, 1)', '(1, 2)', 'trigram_entropy', '(2, 3)', '(2,)', 'unigram_entropy', '(1, 1, 2)

# MI and mRMR

## 10 Percentile

In [20]:
lr = LogisticRegression()
params = {'C': [1, 10, 100, 1000], 'max_iter': [1000, 2000, 5000, 10000]}
accuracy_list_lr_10_mi_mrmr, f1_score_list_lr_10_mi_mrmr, auc_list_lr_10_mi_mrmr, param_list_lr_10_mi_mrmr = model_train_predict(lr, 'mi_mrmr_feat_list_10*', params=params)
print("\n================================================================\n")
rfc = RandomForestClassifier()
params = {'n_estimators': [10, 50, 100], 'max_depth': [3, 5, 10]}
accuracy_list_rfc_10_mi_mrmr, f1_score_list_rfc_10_mi_mrmr, auc_list_rfc_10_mi_mrmr, param_list_rfc_10_mi_mrmr = model_train_predict(rfc, 'mi_mrmr_feat_list_10', params=params)
print("\n================================================================\n")
svc = SVC()
params = {'C': [ 1, 10, 100], 'kernel': ['linear', 'poly', 'rbf', 'sigmoid'], 'gamma': ['auto', 'scale'],}
accuracy_list_svm_10_mi_mrmr, f1_score_list_svm_10_mi_mrmr, auc_list_svm_10_mi_mrmr, param_list_svm_10_mi_mrmr = model_train_predict(svc, 'mi_mrmr_feat_list_10', params=params)
print("\n================================================================\n")
xgbc = xgb.XGBClassifier()
params = {'n_estimators': [10, 50, 100], 'max_depth': [3, 5, 10]}
accuracy_list_xgb_10_mi_mrmr, f1_score_list_xgb_10_mi_mrmr, auc_list_xgb_10_mi_mrmr, param_list_xgb_10_mi_mrmr = model_train_predict(xgbc, 'mi_mrmr_feat_list_10', params=params)
print("\n================================================================\n")
lgbc = lgb.LGBMClassifier()
params = {'learning_rate': [0.1, 0.05, 0.01], 'n_estimators': [10, 50, 100], 'max_depth': [3, 5, 10]}
accuracy_list_lgb_10_mi_mrmr, f1_score_list_lgb_10_mi_mrmr, auc_list_lgb_10_mi_mrmr, param_list_lgb_10_mi_mrmr = model_train_predict(lgbc, 'mi_mrmr_feat_list_10', params=params)

(12052, 11) (12052,) (3014, 11) (3014,)
Average Accuracy 0.8340743198407432
Average F1 Score 0.8306314834669328
Average AUC 0.8340743198407432
Max Accuracy 0.8420703384207033
Max F1 Score 0.8376534788540245
Max AUC 0.8420703384207034
[0.8311214333112144, 0.8420703384207033, 0.8367617783676178, 0.8321167883211679, 0.8258128732581287, 0.8390842733908427, 0.8205043132050431, 0.8374253483742535, 0.8394160583941606, 0.8364299933642999]
[0.8311214333112144, 0.8420703384207034, 0.8367617783676178, 0.8321167883211679, 0.8258128732581288, 0.8390842733908427, 0.8205043132050431, 0.8374253483742534, 0.8394160583941604, 0.8364299933643]
Best Sample Index based on Max Accuracy 1
Best Sample Index based on Max F1 Score 1
Best Sample Index based on Max AUC 1
Best Features based on Max Accuracy ['bigram_entropy', '(1, 2, 1)', 'unigram_entropy', '(3, 1, 1)', '(2, 3)', '(2,)', '(1, 2)', '(3, 1)', '(2, 1)', 'trigram_entropy']
Best Features based on Max F1 Score ['bigram_entropy', '(1, 2, 1)', 'unigram_en

## 20 Percentile

In [21]:
lr = LogisticRegression()
params = {'C': [1, 10, 100, 1000], 'max_iter': [1000, 2000, 5000, 10000]}
accuracy_list_lr_20_mi_mrmr, f1_score_list_lr_20_mi_mrmr, auc_list_lr_20_mi_mrmr, param_list_lr_20_mi_mrmr = model_train_predict(lr, 'mi_mrmr_feat_list_20*', params=params)
print("\n================================================================\n")
rfc = RandomForestClassifier()
params = {'n_estimators': [10, 50, 100], 'max_depth': [3, 5, 10]}
accuracy_list_rfc_20_mi_mrmr, f1_score_list_rfc_20_mi_mrmr, auc_list_rfc_20_mi_mrmr, param_list_rfc_20_mi_mrmr = model_train_predict(rfc, 'mi_mrmr_feat_list_20', params=params)
print("\n================================================================\n")
svc = SVC()
params = {'C': [ 1, 10, 100], 'kernel': ['linear', 'poly', 'rbf', 'sigmoid'], 'gamma': ['auto', 'scale'],}
accuracy_list_svm_20_mi_mrmr, f1_score_list_svm_20_mi_mrmr, auc_list_svm_20_mi_mrmr, param_list_svm_20_mi_mrmr = model_train_predict(svc, 'mi_mrmr_feat_list_20', params=params)
print("\n================================================================\n")
xgbc = xgb.XGBClassifier()
params = {'n_estimators': [10, 50, 100], 'max_depth': [3, 5, 10]}
accuracy_list_xgb_20_mi_mrmr, f1_score_list_xgb_20_mi_mrmr, auc_list_xgb_20_mi_mrmr, param_list_xgb_20_mi_mrmr = model_train_predict(xgbc, 'mi_mrmr_feat_list_20', params=params)
print("\n================================================================\n")
lgbc = lgb.LGBMClassifier()
params = {'learning_rate': [0.1, 0.05, 0.01], 'n_estimators': [10, 50, 100], 'max_depth': [3, 5, 10]}
accuracy_list_lgb_20_mi_mrmr, f1_score_list_lgb_20_mi_mrmr, auc_list_lgb_20_mi_mrmr, param_list_lgb_20_mi_mrmr = model_train_predict(lgbc, 'mi_mrmr_feat_list_20', params=params)

(12052, 24) (12052,) (3014, 24) (3014,)
Average Accuracy 0.863437292634373
Average F1 Score 0.8624104088952919
Average AUC 0.863437292634373
Max Accuracy 0.873921698739217
Max F1 Score 0.8717948717948719
Max AUC 0.8739216987392171
[0.8613138686131386, 0.8679495686794957, 0.8573324485733245, 0.8576642335766423, 0.858327803583278, 0.8626410086264101, 0.8626410086264101, 0.873921698739217, 0.8619774386197744, 0.8706038487060385]
[0.8613138686131386, 0.8679495686794957, 0.8573324485733247, 0.8576642335766422, 0.858327803583278, 0.86264100862641, 0.8626410086264101, 0.8739216987392171, 0.8619774386197745, 0.8706038487060385]
Best Sample Index based on Max Accuracy 7
Best Sample Index based on Max F1 Score 7
Best Sample Index based on Max AUC 7
Best Features based on Max Accuracy ['(1, 4)', 'unigram_entropy', '(4,)', '(2, 3)', '(3, 3)', '(1, 2, 1)', '(1, 1, 2)', '(1, 2, 3)', '(4, 1, 1)', '(4, 1)', 'pattern_hvg_4_nodes_entropy', '(1, 2)', '(2, 3, 1)', '(3, 1)', 'bigram_entropy', '(3,)', '(1, 

## 30 Percentile

In [22]:
lr = LogisticRegression()
params = {'C': [1, 10, 100, 1000], 'max_iter': [1000, 2000, 5000, 10000]}
accuracy_list_lr_30_mi_mrmr, f1_score_list_lr_30_mi_mrmr, auc_list_lr_30_mi_mrmr, param_list_lr_30_mi_mrmr = model_train_predict(lr, 'mi_mrmr_feat_list_30*', params=params)
print("\n================================================================\n")
rfc = RandomForestClassifier()
params = {'n_estimators': [10, 50, 100], 'max_depth': [3, 5, 10]}
accuracy_list_rfc_30_mi_mrmr, f1_score_list_rfc_30_mi_mrmr, auc_list_rfc_30_mi_mrmr, param_list_rfc_30_mi_mrmr = model_train_predict(rfc, 'mi_mrmr_feat_list_30', params=params)
print("\n================================================================\n")
svc = SVC()
params = {'C': [ 1, 10, 100], 'kernel': ['linear', 'poly', 'rbf', 'sigmoid'], 'gamma': ['auto', 'scale'],}
accuracy_list_svm_30_mi_mrmr, f1_score_list_svm_30_mi_mrmr, auc_list_svm_30_mi_mrmr, param_list_svm_30_mi_mrmr = model_train_predict(svc, 'mi_mrmr_feat_list_30', params=params)
print("\n================================================================\n")
xgbc = xgb.XGBClassifier()
params = {'n_estimators': [10, 50, 100], 'max_depth': [3, 5, 10]}
accuracy_list_xgb_30_mi_mrmr, f1_score_list_xgb_30_mi_mrmr, auc_list_xgb_30_mi_mrmr, param_list_xgb_30_mi_mrmr = model_train_predict(xgbc, 'mi_mrmr_feat_list_30', params=params)
print("\n================================================================\n")
lgbc = lgb.LGBMClassifier()
params = {'learning_rate': [0.1, 0.05, 0.01], 'n_estimators': [10, 50, 100], 'max_depth': [3, 5, 10]}
accuracy_list_lgb_30_mi_mrmr, f1_score_list_lgb_30_mi_mrmr, auc_list_lgb_30_mi_mrmr, param_list_lgb_30_mi_mrmr = model_train_predict(lgbc, 'mi_mrmr_feat_list_30', params=params)

/Users/nitanshjain/.local/share/virtualenvs/shopper-intent-prediction-L5e-nQUd/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/nitanshjain/.local/share/virtualenvs/shopper-intent-prediction-L5e-nQUd/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documen

(12052, 30) (12052,) (3014, 30) (3014,)
Average Accuracy 0.8714996682149966
Average F1 Score 0.8709317005276673
Average AUC 0.8714996682149966
Max Accuracy 0.8775713337757134
Max F1 Score 0.8757157292017514
Max AUC 0.8775713337757134
[0.8656270736562708, 0.8752488387524884, 0.8729263437292635, 0.8633045786330458, 0.8729263437292635, 0.8672859986728599, 0.8702720637027206, 0.8775713337757134, 0.8749170537491705, 0.8749170537491705]
[0.8656270736562708, 0.8752488387524885, 0.8729263437292635, 0.8633045786330458, 0.8729263437292634, 0.8672859986728599, 0.8702720637027206, 0.8775713337757134, 0.8749170537491704, 0.8749170537491704]
Best Sample Index based on Max Accuracy 7
Best Sample Index based on Max F1 Score 7
Best Sample Index based on Max AUC 7
Best Features based on Max Accuracy ['(1, 4)', 'unigram_entropy', '(4,)', '(2, 3)', '(3, 3, 1)', '(3, 3, 3)', '(3, 3)', '(1, 2, 1)', '(1, 1, 2)', '(1, 2, 3)', '(4, 1, 1)', '(4, 1)', 'pattern_hvg_4_nodes_entropy', '(1, 2)', '(2, 3, 1)', '(3, 1)

## 50 Percentile

In [23]:
lr = LogisticRegression()
params = {'C': [1, 10, 100, 1000], 'max_iter': [1000, 2000, 5000, 10000]}
accuracy_list_lr_50_mi_mrmr, f1_score_list_lr_50_mi_mrmr, auc_list_lr_50_mi_mrmr, param_list_lr_50_mi_mrmr = model_train_predict(lr, 'mi_mrmr_feat_list_50*', params=params)
print("\n================================================================\n")
rfc = RandomForestClassifier()
params = {'n_estimators': [10, 50, 100], 'max_depth': [3, 5, 10]}
accuracy_list_rfc_50_mi_mrmr, f1_score_list_rfc_50_mi_mrmr, auc_list_rfc_50_mi_mrmr, param_list_rfc_50_mi_mrmr = model_train_predict(rfc, 'mi_mrmr_feat_list_50', params=params)
print("\n================================================================\n")
svc = SVC()
params = {'C': [ 1, 10, 100], 'kernel': ['linear', 'poly', 'rbf', 'sigmoid'], 'gamma': ['auto', 'scale'],}
accuracy_list_svm_50_mi_mrmr, f1_score_list_svm_50_mi_mrmr, auc_list_svm_50_mi_mrmr, param_list_svm_50_mi_mrmr = model_train_predict(svc, 'mi_mrmr_feat_list_50', params=params)
print("\n================================================================\n")
xgbc = xgb.XGBClassifier()
params = {'n_estimators': [10, 50, 100], 'max_depth': [3, 5, 10]}
accuracy_list_xgb_50_mi_mrmr, f1_score_list_xgb_50_mi_mrmr, auc_list_xgb_50_mi_mrmr, param_list_xgb_50_mi_mrmr = model_train_predict(xgbc, 'mi_mrmr_feat_list_50', params=params)
print("\n================================================================\n")
lgbc = lgb.LGBMClassifier()
params = {'learning_rate': [0.1, 0.05, 0.01], 'n_estimators': [10, 50, 100], 'max_depth': [3, 5, 10]}
accuracy_list_lgb_50_mi_mrmr, f1_score_list_lgb_50_mi_mrmr, auc_list_lgb_50_mi_mrmr, param_list_lgb_50_mi_mrmr = model_train_predict(lgbc, 'mi_mrmr_feat_list_50', params=params)

/Users/nitanshjain/.local/share/virtualenvs/shopper-intent-prediction-L5e-nQUd/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/nitanshjain/.local/share/virtualenvs/shopper-intent-prediction-L5e-nQUd/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documen

(12052, 56) (12052,) (3014, 56) (3014,)
Average Accuracy 0.8767086927670871
Average F1 Score 0.8762207619683811
Average AUC 0.8767086927670869
Max Accuracy 0.8842070338420703
Max F1 Score 0.8824520040417649
Max AUC 0.8842070338420703
[0.8742534837425349, 0.8825481088254811, 0.8762441937624419, 0.8709356337093563, 0.8732581287325812, 0.8729263437292635, 0.8699402786994028, 0.8842070338420703, 0.8822163238221632, 0.880557398805574]
[0.8742534837425348, 0.8825481088254811, 0.8762441937624419, 0.8709356337093566, 0.8732581287325812, 0.8729263437292634, 0.8699402786994027, 0.8842070338420703, 0.8822163238221632, 0.8805573988055739]
Best Sample Index based on Max Accuracy 7
Best Sample Index based on Max F1 Score 7
Best Sample Index based on Max AUC 7
Best Features based on Max Accuracy ['(1, 4)', 'unigram_entropy', '(1, 4, 4)', '(3, 2, 3)', '(4,)', '(2, 3, 2)', '(2, 3)', '(3, 3, 1)', '(6, 2, 3)', '(3, 3, 3)', '(6, 3, 1)', '(3, 3)', '(1, 2, 1)', '(1, 1, 2)', '(3, 2, 2)', '(1, 2, 3)', '(4, 1,

## 75 Percentile

In [24]:
lr = LogisticRegression()
params = {'C': [1, 10, 100, 1000], 'max_iter': [1000, 2000, 5000, 10000]}
accuracy_list_lr_75_mi_mrmr, f1_score_list_lr_75_mi_mrmr, auc_list_lr_75_mi_mrmr, param_list_lr_75_mi_mrmr = model_train_predict(lr, 'mi_mrmr_feat_list_75', params=params)
print("\n================================================================\n")
rfc = RandomForestClassifier()
params = {'n_estimators': [10, 50, 100], 'max_depth': [3, 5, 10]}
accuracy_list_rfc_75_mi_mrmr, f1_score_list_rfc_75_mi_mrmr, auc_list_rfc_75_mi_mrmr, param_list_rfc_75_mi_mrmr = model_train_predict(rfc, 'mi_mrmr_feat_list_75', params=params)
print("\n================================================================\n")
svc = SVC()
params = {'C': [ 1, 10, 100], 'kernel': ['linear', 'poly', 'rbf', 'sigmoid'], 'gamma': ['auto', 'scale'],}
accuracy_list_svm_75_mi_mrmr, f1_score_list_svm_75_mi_mrmr, auc_list_svm_75_mi_mrmr, param_list_svm_75_mi_mrmr = model_train_predict(svc, 'mi_mrmr_feat_list_75', params=params)
print("\n================================================================\n")
xgbc = xgb.XGBClassifier()
params = {'n_estimators': [10, 50, 100], 'max_depth': [3, 5, 10]}
accuracy_list_xgb_75_mi_mrmr, f1_score_list_xgb_75_mi_mrmr, auc_list_xgb_75_mi_mrmr, param_list_xgb_75_mi_mrmr = model_train_predict(xgbc, 'mi_mrmr_feat_list_75', params=params)
print("\n================================================================\n")
lgbc = lgb.LGBMClassifier()
params = {'learning_rate': [0.1, 0.05, 0.01], 'n_estimators': [10, 50, 100], 'max_depth': [3, 5, 10]}
accuracy_list_lgb_75_mi_mrmr, f1_score_list_lgb_75_mi_mrmr, auc_list_lgb_75_mi_mrmr, param_list_lgb_75_mi_mrmr = model_train_predict(lgbc, 'mi_mrmr_feat_list_75', params=params)

/Users/nitanshjain/.local/share/virtualenvs/shopper-intent-prediction-L5e-nQUd/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/nitanshjain/.local/share/virtualenvs/shopper-intent-prediction-L5e-nQUd/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documen

(12052, 88) (12052,) (3014, 88) (3014,)
Average Accuracy 0.8792302587923025
Average F1 Score 0.878954055390724
Average AUC 0.8792302587923025
Max Accuracy 0.8861977438619775
Max F1 Score 0.8846283215607131
Max AUC 0.8861977438619775
[0.8798938287989383, 0.8848706038487061, 0.882879893828799, 0.8742534837425349, 0.8692767086927671, 0.8788984737889848, 0.8729263437292635, 0.8861977438619775, 0.8815527538155276, 0.8815527538155276]
[0.8798938287989383, 0.884870603848706, 0.8828798938287991, 0.8742534837425349, 0.8692767086927671, 0.8788984737889847, 0.8729263437292635, 0.8861977438619775, 0.8815527538155276, 0.8815527538155276]
Best Sample Index based on Max Accuracy 7
Best Sample Index based on Max F1 Score 7
Best Sample Index based on Max AUC 7
Best Features based on Max Accuracy ['(1, 4, 4)', '(1, 6, 2)', '(4,)', 'B4', '(6, 3, 1)', 'C5', '(1, 1, 2)', '(2, 3, 1)', '(3, 1)', '(2, 3, 3)', '(3,)', '(2, 2, 3)', '(1, 4, 1)', '(4, 4)', '(3, 1, 2)', '(3, 1, 3)', '(1, 2, 6)', '(4, 4, 1)', '(2, 

## 90 Percentile

In [25]:
lr = LogisticRegression()
params = {'C': [1, 10, 100, 1000], 'max_iter': [1000, 2000, 5000, 10000]}
accuracy_list_lr_90_mi_mrmr, f1_score_list_lr_90_mi_mrmr, auc_list_lr_90_mi_mrmr, param_list_lr_90_mi_mrmr = model_train_predict(lr, 'mi_mrmr_feat_list_90', params=params)
print("\n================================================================\n")
rfc = RandomForestClassifier()
params = {'n_estimators': [10, 50, 100], 'max_depth': [3, 5, 10]}
accuracy_list_rfc_90_mi_mrmr, f1_score_list_rfc_90_mi_mrmr, auc_list_rfc_90_mi_mrmr, param_list_rfc_90_mi_mrmr = model_train_predict(rfc, 'mi_mrmr_feat_list_90', params=params)
print("\n================================================================\n")
svc = SVC()
params = {'C': [ 1, 10, 100], 'kernel': ['linear', 'poly', 'rbf', 'sigmoid'], 'gamma': ['auto', 'scale'],}
accuracy_list_svm_90_mi_mrmr, f1_score_list_svm_90_mi_mrmr, auc_list_svm_90_mi_mrmr, param_list_svm_90_mi_mrmr = model_train_predict(svc, 'mi_mrmr_feat_list_90', params=params)
print("\n================================================================\n")
xgbc = xgb.XGBClassifier()
params = {'n_estimators': [10, 50, 100], 'max_depth': [3, 5, 10]}
accuracy_list_xgb_90_mi_mrmr, f1_score_list_xgb_90_mi_mrmr, auc_list_xgb_90_mi_mrmr, param_list_xgb_90_mi_mrmr = model_train_predict(xgbc, 'mi_mrmr_feat_list_90', params=params)
print("\n================================================================\n")
lgbc = lgb.LGBMClassifier()
params = {'learning_rate': [0.1, 0.05, 0.01], 'n_estimators': [10, 50, 100], 'max_depth': [3, 5, 10]}
accuracy_list_lgb_90_mi_mrmr, f1_score_list_lgb_90_mi_mrmr, auc_list_lgb_90_mi_mrmr, param_list_lgb_90_mi_mrmr = model_train_predict(lgbc, 'mi_mrmr_feat_list_90', params=params)

/Users/nitanshjain/.local/share/virtualenvs/shopper-intent-prediction-L5e-nQUd/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/nitanshjain/.local/share/virtualenvs/shopper-intent-prediction-L5e-nQUd/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documen

(12052, 117) (12052,) (3014, 117) (3014,)
Average Accuracy 0.8810550763105507
Average F1 Score 0.8807647285548736
Average AUC 0.8810550763105507
Max Accuracy 0.8881884538818845
Max F1 Score 0.8864937689457729
Max AUC 0.8881884538818845
[0.8808891838088918, 0.8855341738553417, 0.8832116788321168, 0.8759124087591241, 0.8706038487060385, 0.8802256138022562, 0.8792302587923025, 0.8881884538818845, 0.882879893828799, 0.8838752488387525]
[0.8808891838088918, 0.8855341738553417, 0.8832116788321169, 0.875912408759124, 0.8706038487060384, 0.8802256138022561, 0.8792302587923025, 0.8881884538818845, 0.8828798938287991, 0.8838752488387525]
Best Sample Index based on Max Accuracy 7
Best Sample Index based on Max F1 Score 7
Best Sample Index based on Max AUC 7
Best Features based on Max Accuracy ['(6, 1)', '(1, 4, 4)', '(1, 6, 2)', '(4,)', '(6, 2)', '(6, 3, 1)', 'C5', '(1, 1, 2)', '(1, 3, 2)', 'M5', '(2, 3, 1)', '(3, 1)', '(2, 3, 3)', '(3,)', '(2, 2, 3)', '(1, 4, 1)', '(4, 4)', 'U5', '(3, 1, 2)', '(

# PCA

In [26]:
def model_train_predict_pca(model, k, dataframes=list_sample_dataframes, params=None):
    
    accuracy_list = []
    f1_score_list = []
    auc_list = []
    param_list = []
    
    for sample in dataframes:
        x = sample.drop(['Unnamed: 0', 'conversion_class'], axis=1)
        y = sample['conversion_class']
        x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42, stratify=y)
        
        pca = PCA(n_components=k)
        x_train = pca.fit_transform(x_train)
        x_test = pca.transform(x_test)
        
        clf = GridSearchCV(estimator=model, param_grid=params, cv=5, n_jobs=-1)
        clf.fit(x_train, y_train)
        y_pred = clf.predict(x_test)
        # model.fit(x_train, y_train)
        # y_pred = model.predict(x_test)
        accuracy_list.append(accuracy_score(y_test, y_pred))
        f1_score_list.append(f1_score(y_test, y_pred))
        auc_list.append(roc_auc_score(y_test, y_pred))
        param_list.append(clf.best_params_)

    print('Average Accuracy', np.mean(accuracy_list))
    print('Average F1 Score', np.mean(f1_score_list))
    print('Average AUC', np.mean(auc_list)) 
    
    print('Max Accuracy', max(accuracy_list))
    print('Max F1 Score', max(f1_score_list))
    print('Max AUC', max(auc_list))  
    
    best_accuracy_index = accuracy_list.index(max(accuracy_list))
    best_f1_score_index = f1_score_list.index(max(f1_score_list))
    best_auc_index = auc_list.index(max(auc_list))
    
    print('Best Sample Index based on Max Accuracy', best_accuracy_index)
    print('Best Sample Index based on Max F1 Score', best_f1_score_index)
    print('Best Sample Index based on Max AUC', best_auc_index)
    print('Best Parameters', param_list[best_accuracy_index])
    
    return accuracy_list, f1_score_list, auc_list, param_list  

## 10 Percentile

In [27]:
lr = LogisticRegression()
params = {'C': [1, 10, 100, 1000], 'max_iter': [1000, 2000, 5000, 10000]}
accuracy_list_lr_10_pca, f1_score_list_lr_10_pca, auc_list_lr_10_pca, param_list_lr_10_pca = model_train_predict_pca(lr, 14, params=params)
print("\n================================================================\n")
rfc = RandomForestClassifier()
params = {'n_estimators': [10, 50, 100], 'max_depth': [3, 5, 10]}
accuracy_list_rfc_10_pca, f1_score_list_rfc_10_pca, auc_list_rfc_10_pca, param_list_rfc_10_pca = model_train_predict_pca(rfc, 14, params=params)
print("\n================================================================\n")
svc = SVC()
params = {'C': [ 1, 10, 100], 'kernel': ['linear', 'poly', 'rbf', 'sigmoid'], 'gamma': ['auto', 'scale'],}
accuracy_list_svm_10_pca, f1_score_list_svm_10_pca, auc_list_svm_10_pca, param_list_svm_10_pca = model_train_predict_pca(svc, 14, params=params)
print("\n================================================================\n")
xgbc = xgb.XGBClassifier()
params = {'n_estimators': [10, 50, 100], 'max_depth': [3, 5, 10]}
accuracy_list_xgb_10_pca, f1_score_list_xgb_10_pca, auc_list_xgb_10_pca, param_list_xgb_10_pca = model_train_predict_pca(xgbc, 14, params=params)
print("\n================================================================\n")
lgbc = lgb.LGBMClassifier()
params = {'learning_rate': [0.1, 0.05, 0.01], 'n_estimators': [10, 50, 100], 'max_depth': [3, 5, 10]}
accuracy_list_lgb_10_pca, f1_score_list_lgb_10_pca, auc_list_lgb_10_pca, param_list_lgb_10_pca = model_train_predict_pca(lgbc, 14, params=params)

Average Accuracy 0.8215265486725662
Average F1 Score 0.8225302251357952
Average AUC 0.8215265486725662
Max Accuracy 0.8265486725663717
Max F1 Score 0.8273888154997798
Max AUC 0.8265486725663717
Best Sample Index based on Max Accuracy 7
Best Sample Index based on Max F1 Score 7
Best Sample Index based on Max AUC 8
Best Parameters {'C': 100, 'max_iter': 1000}


Average Accuracy 0.8345132743362832
Average F1 Score 0.8401911839515765
Average AUC 0.8345132743362832
Max Accuracy 0.8398230088495575
Max F1 Score 0.8444539798326539
Max AUC 0.8398230088495575
Best Sample Index based on Max Accuracy 5
Best Sample Index based on Max F1 Score 7
Best Sample Index based on Max AUC 5
Best Parameters {'max_depth': 10, 'n_estimators': 50}


Average Accuracy 0.8423893805309735
Average F1 Score 0.8488485632587794
Average AUC 0.8423893805309733
Max Accuracy 0.8482300884955752
Max F1 Score 0.8540425531914894
Max AUC 0.8482300884955751
Best Sample Index based on Max Accuracy 5
Best Sample Index based on Max 

## 20 Percentile

In [28]:
lr = LogisticRegression()
params = {'C': [1, 10, 100, 1000], 'max_iter': [1000, 2000, 5000, 10000]}
accuracy_list_lr_20_pca, f1_score_list_lr_20_pca, auc_list_lr_20_pca, param_list_lr_20_pca = model_train_predict_pca(lr, 28, params=params)
print("\n================================================================\n")
rfc = RandomForestClassifier()
params = {'n_estimators': [10, 50, 100], 'max_depth': [3, 5, 10]}
accuracy_list_rfc_20_pca, f1_score_list_rfc_20_pca, auc_list_rfc_20_pca, param_list_rfc_20_pca = model_train_predict_pca(rfc, 28, params=params)
print("\n================================================================\n")
svc = SVC()
params = {'C': [ 1, 10, 100], 'kernel': ['linear', 'poly', 'rbf', 'sigmoid'], 'gamma': ['auto', 'scale'],}
accuracy_list_svm_20_pca, f1_score_list_svm_20_pca, auc_list_svm_20_pca, param_list_svm_20_pca = model_train_predict_pca(svc, 28, params=params)
print("\n================================================================\n")
xgbc = xgb.XGBClassifier()
params = {'n_estimators': [10, 50, 100], 'max_depth': [3, 5, 10]}
accuracy_list_xgb_20_pca, f1_score_list_xgb_20_pca, auc_list_xgb_20_pca, param_list_xgb_20_pca = model_train_predict_pca(xgbc, 28, params=params)
print("\n================================================================\n")
lgbc = lgb.LGBMClassifier()
params = {'learning_rate': [0.1, 0.05, 0.01], 'n_estimators': [10, 50, 100], 'max_depth': [3, 5, 10]}
accuracy_list_lgb_20_pca, f1_score_list_lgb_20_pca, auc_list_lgb_20_pca, param_list_lgb_20_pca = model_train_predict_pca(lgbc, 28, params=params)

Average Accuracy 0.8486725663716814
Average F1 Score 0.8472697529473285
Average AUC 0.8486725663716813
Max Accuracy 0.8553097345132743
Max F1 Score 0.8535602328705778
Max AUC 0.8553097345132743
Best Sample Index based on Max Accuracy 7
Best Sample Index based on Max F1 Score 7
Best Sample Index based on Max AUC 7
Best Parameters {'C': 1, 'max_iter': 1000}


Average Accuracy 0.8563274336283186
Average F1 Score 0.859230173028711
Average AUC 0.8563274336283186
Max Accuracy 0.8630530973451327
Max F1 Score 0.8648176457741866
Max AUC 0.8630530973451327
Best Sample Index based on Max Accuracy 5
Best Sample Index based on Max F1 Score 5
Best Sample Index based on Max AUC 5
Best Parameters {'max_depth': 10, 'n_estimators': 100}


Average Accuracy 0.8715929203539824
Average F1 Score 0.8746482839671078
Average AUC 0.8715929203539824
Max Accuracy 0.879646017699115
Max F1 Score 0.88209796272215
Max AUC 0.8796460176991151
Best Sample Index based on Max Accuracy 7
Best Sample Index based on Max F1 Sc

## 30 Percentile

In [29]:
lr = LogisticRegression()
params = {'C': [1, 10, 100, 1000], 'max_iter': [1000, 2000, 5000, 10000]}
accuracy_list_lr_30_pca, f1_score_list_lr_30_pca, auc_list_lr_30_pca, param_list_lr_30_pca = model_train_predict_pca(lr, 42, params=params)
print("\n================================================================\n")
rfc = RandomForestClassifier()
params = {'n_estimators': [10, 50, 100], 'max_depth': [3, 5, 10]}
accuracy_list_rfc_30_pca, f1_score_list_rfc_30_pca, auc_list_rfc_30_pca, param_list_rfc_30_pca = model_train_predict_pca(rfc, 42, params=params)
print("\n================================================================\n")
svc = SVC()
params = {'C': [ 1, 10, 100], 'kernel': ['linear', 'poly', 'rbf', 'sigmoid'], 'gamma': ['auto', 'scale'],}
accuracy_list_svm_30_pca, f1_score_list_svm_30_pca, auc_list_svm_30_pca, param_list_svm_30_pca = model_train_predict_pca(svc, 42, params=params)
print("\n================================================================\n")
xgbc = xgb.XGBClassifier()
params = {'n_estimators': [10, 50, 100], 'max_depth': [3, 5, 10]}
accuracy_list_xgb_30_pca, f1_score_list_xgb_30_pca, auc_list_xgb_30_pca, param_list_xgb_30_pca = model_train_predict_pca(xgbc, 42, params=params)
print("\n================================================================\n")
lgbc = lgb.LGBMClassifier()
params = {'learning_rate': [0.1, 0.05, 0.01], 'n_estimators': [10, 50, 100], 'max_depth': [3, 5, 10]}
accuracy_list_lgb_30_pca, f1_score_list_lgb_30_pca, auc_list_lgb_30_pca, param_list_lgb_30_pca = model_train_predict_pca(lgbc, 42, params=params)

Average Accuracy 0.8706415929203539
Average F1 Score 0.8699413689950951
Average AUC 0.870641592920354
Max Accuracy 0.8778761061946903
Max F1 Score 0.8761776581426649
Max AUC 0.8778761061946903
Best Sample Index based on Max Accuracy 7
Best Sample Index based on Max F1 Score 7
Best Sample Index based on Max AUC 7
Best Parameters {'C': 1000, 'max_iter': 1000}


Average Accuracy 0.8801991150442479
Average F1 Score 0.8828696631245334
Average AUC 0.8801991150442477
Max Accuracy 0.8880530973451327
Max F1 Score 0.8900955690703736
Max AUC 0.8880530973451328
Best Sample Index based on Max Accuracy 7
Best Sample Index based on Max F1 Score 7
Best Sample Index based on Max AUC 7
Best Parameters {'max_depth': 10, 'n_estimators': 100}


Average Accuracy 0.8869911504424778
Average F1 Score 0.8885193722812271
Average AUC 0.8869911504424778
Max Accuracy 0.8924778761061947
Max F1 Score 0.8939327804452204
Max AUC 0.8924778761061947
Best Sample Index based on Max Accuracy 5
Best Sample Index based on Max

## 50 Percentile

In [30]:
lr = LogisticRegression()
params = {'C': [1, 10, 100, 1000], 'max_iter': [1000, 2000, 5000, 10000]}
accuracy_list_lr_50_pca, f1_score_list_lr_50_pca, auc_list_lr_50_pca, param_list_lr_50_pca = model_train_predict_pca(lr, 69, params=params)
print("\n================================================================\n")
rfc = RandomForestClassifier()
params = {'n_estimators': [10, 50, 100], 'max_depth': [3, 5, 10]}
accuracy_list_rfc_50_pca, f1_score_list_rfc_50_pca, auc_list_rfc_50_pca, param_list_rfc_50_pca = model_train_predict_pca(rfc, 69, params=params)
print("\n================================================================\n")
svc = SVC()
params = {'C': [ 1, 10, 100], 'kernel': ['linear', 'poly', 'rbf', 'sigmoid'], 'gamma': ['auto', 'scale'],}
accuracy_list_svm_50_pca, f1_score_list_svm_50_pca, auc_list_svm_50_pca, param_list_svm_50_pca = model_train_predict_pca(svc, 69, params=params)
print("\n================================================================\n")
xgbc = xgb.XGBClassifier()
params = {'n_estimators': [10, 50, 100], 'max_depth': [3, 5, 10]}
accuracy_list_xgb_50_pca, f1_score_list_xgb_50_pca, auc_list_xgb_50_pca, param_list_xgb_50_pca = model_train_predict_pca(xgbc, 69, params=params)
print("\n================================================================\n")
lgbc = lgb.LGBMClassifier()
params = {'learning_rate': [0.1, 0.05, 0.01], 'n_estimators': [10, 50, 100], 'max_depth': [3, 5, 10]}
accuracy_list_lgb_50_pca, f1_score_list_lgb_50_pca, auc_list_lgb_50_pca, param_list_lgb_50_pca = model_train_predict_pca(lgbc, 69, params=params)

/Users/nitanshjain/.local/share/virtualenvs/shopper-intent-prediction-L5e-nQUd/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/nitanshjain/.local/share/virtualenvs/shopper-intent-prediction-L5e-nQUd/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documen

Average Accuracy 0.8787831858407079
Average F1 Score 0.8786559857490769
Average AUC 0.8787831858407079
Max Accuracy 0.8823008849557522
Max F1 Score 0.8811375249944458
Max AUC 0.8823008849557522
Best Sample Index based on Max Accuracy 7
Best Sample Index based on Max F1 Score 8
Best Sample Index based on Max AUC 7
Best Parameters {'C': 1000, 'max_iter': 1000}


Average Accuracy 0.8860619469026549
Average F1 Score 0.888516915326931
Average AUC 0.8860619469026549
Max Accuracy 0.8924778761061947
Max F1 Score 0.8948961937716262
Max AUC 0.8924778761061946
Best Sample Index based on Max Accuracy 2
Best Sample Index based on Max F1 Score 2
Best Sample Index based on Max AUC 2
Best Parameters {'max_depth': 10, 'n_estimators': 100}


Average Accuracy 0.892433628318584
Average F1 Score 0.8939756069681268
Average AUC 0.892433628318584
Max Accuracy 0.8960176991150443
Max F1 Score 0.8974694589877836
Max AUC 0.8960176991150443
Best Sample Index based on Max Accuracy 2
Best Sample Index based on Max F

## 75 Percentile

In [31]:
lr = LogisticRegression()
params = {'C': [1, 10, 100, 1000], 'max_iter': [1000, 2000, 5000, 10000]}
accuracy_list_lr_75_pca, f1_score_list_lr_75_pca, auc_list_lr_75_pca, param_list_lr_75_pca = model_train_predict_pca(lr, 104, params=params)
print("\n================================================================\n")
rfc = RandomForestClassifier()
params = {'n_estimators': [10, 50, 100], 'max_depth': [3, 5, 10]}
accuracy_list_rfc_75_pca, f1_score_list_rfc_75_pca, auc_list_rfc_75_pca, param_list_rfc_75_pca = model_train_predict_pca(rfc, 104, params=params)
print("\n================================================================\n")
svc = SVC()
params = {'C': [ 1, 10, 100], 'kernel': ['linear', 'poly', 'rbf', 'sigmoid'], 'gamma': ['auto', 'scale'],}
accuracy_list_svm_75_pca, f1_score_list_svm_75_pca, auc_list_svm_75_pca, param_list_svm_75_pca = model_train_predict_pca(svc, 104, params=params)
print("\n================================================================\n")
xgbc = xgb.XGBClassifier()
params = {'n_estimators': [10, 50, 100], 'max_depth': [3, 5, 10]}
accuracy_list_xgb_75_pca, f1_score_list_xgb_75_pca, auc_list_xgb_75_pca, param_list_xgb_75_pca = model_train_predict_pca(xgbc, 104, params=params)
print("\n================================================================\n")
lgbc = lgb.LGBMClassifier()
params = {'learning_rate': [0.1, 0.05, 0.01], 'n_estimators': [10, 50, 100], 'max_depth': [3, 5, 10]}
accuracy_list_lgb_75_pca, f1_score_list_lgb_75_pca, auc_list_lgb_75_pca, param_list_lgb_75_pca = model_train_predict_pca(lgbc, 104, params=params)

/Users/nitanshjain/.local/share/virtualenvs/shopper-intent-prediction-L5e-nQUd/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/nitanshjain/.local/share/virtualenvs/shopper-intent-prediction-L5e-nQUd/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documen

Average Accuracy 0.8807743362831857
Average F1 Score 0.880515341802895
Average AUC 0.8807743362831857
Max Accuracy 0.8853982300884956
Max F1 Score 0.8847864768683275
Max AUC 0.8853982300884956
Best Sample Index based on Max Accuracy 8
Best Sample Index based on Max F1 Score 8
Best Sample Index based on Max AUC 8
Best Parameters {'C': 1000, 'max_iter': 2000}


Average Accuracy 0.8887389380530972
Average F1 Score 0.8906517819305233
Average AUC 0.8887389380530972
Max Accuracy 0.8946902654867257
Max F1 Score 0.8959335373852207
Max AUC 0.8946902654867257
Best Sample Index based on Max Accuracy 7
Best Sample Index based on Max F1 Score 7
Best Sample Index based on Max AUC 7
Best Parameters {'max_depth': 10, 'n_estimators': 100}


Average Accuracy 0.8929867256637168
Average F1 Score 0.8944913903917564
Average AUC 0.8929867256637168
Max Accuracy 0.8971238938053098
Max F1 Score 0.8985380754963996
Max AUC 0.8971238938053097
Best Sample Index based on Max Accuracy 2
Best Sample Index based on Max

## 90 Percentile

In [32]:
lr = LogisticRegression()
params = {'C': [1, 10, 100, 1000], 'max_iter': [1000, 2000, 5000, 10000]}
accuracy_list_lr_90_pca, f1_score_list_lr_90_pca, auc_list_lr_90_pca, param_list_lr_90_pca = model_train_predict_pca(lr, 125, params=params)
print("\n================================================================\n")
rfc = RandomForestClassifier()
params = {'n_estimators': [10, 50, 100], 'max_depth': [3, 5, 10]}
accuracy_list_rfc_90_pca, f1_score_list_rfc_90_pca, auc_list_rfc_90_pca, param_list_rfc_90_pca = model_train_predict_pca(rfc, 125, params=params)
print("\n================================================================\n")
svc = SVC()
params = {'C': [ 1, 10, 100], 'kernel': ['linear', 'poly', 'rbf', 'sigmoid'], 'gamma': ['auto', 'scale'],}
accuracy_list_svm_90_pca, f1_score_list_svm_90_pca, auc_list_svm_90_pca, param_list_svm_90_pca = model_train_predict_pca(svc, 125, params=params)
print("\n================================================================\n")
xgbc = xgb.XGBClassifier()
params = {'n_estimators': [10, 50, 100], 'max_depth': [3, 5, 10]}
accuracy_list_xgb_90_pca, f1_score_list_xgb_90_pca, auc_list_xgb_90_pca, param_list_xgb_90_pca = model_train_predict_pca(xgbc, 125, params=params)
print("\n================================================================\n")
lgbc = lgb.LGBMClassifier()
params = {'learning_rate': [0.1, 0.05, 0.01], 'n_estimators': [10, 50, 100], 'max_depth': [3, 5, 10]}
accuracy_list_lgb_90_pca, f1_score_list_lgb_90_pca, auc_list_lgb_90_pca, param_list_lgb_90_pca = model_train_predict_pca(lgbc, 125, params=params)

/Users/nitanshjain/.local/share/virtualenvs/shopper-intent-prediction-L5e-nQUd/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/nitanshjain/.local/share/virtualenvs/shopper-intent-prediction-L5e-nQUd/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documen

Average Accuracy 0.8817699115044249
Average F1 Score 0.8815000037028407
Average AUC 0.8817699115044249
Max Accuracy 0.886283185840708
Max F1 Score 0.8857269897732326
Max AUC 0.886283185840708
Best Sample Index based on Max Accuracy 8
Best Sample Index based on Max F1 Score 8
Best Sample Index based on Max AUC 8
Best Parameters {'C': 1000, 'max_iter': 2000}


Average Accuracy 0.888075221238938
Average F1 Score 0.889869908024887
Average AUC 0.888075221238938
Max Accuracy 0.8973451327433628
Max F1 Score 0.8983347940403155
Max AUC 0.8973451327433628
Best Sample Index based on Max Accuracy 7
Best Sample Index based on Max F1 Score 7
Best Sample Index based on Max AUC 7
Best Parameters {'max_depth': 10, 'n_estimators': 100}


Average Accuracy 0.8929867256637168
Average F1 Score 0.8944866126827042
Average AUC 0.8929867256637166
Max Accuracy 0.8971238938053098
Max F1 Score 0.8985380754963996
Max AUC 0.8971238938053097
Best Sample Index based on Max Accuracy 2
Best Sample Index based on Max F1 

# Saving results

In [40]:
models = ['lr', 'rfc', 'svc', 'xgbc', 'lgbm']
models = [value for value in models for _ in range(10)] * 6
percentiles = ['10', '20', '30', '50', '75', '90']
percentiles = [value for value in percentiles for _ in range(50)]
filename_sample_list_final = filename_sample_list * 30

print(len(models))
print(len(percentiles))
print(len(filename_sample_list_final))

300
300
300


In [41]:
overall_accuracy_list_mi = (accuracy_list_lr_10_mi + accuracy_list_rfc_10_mi + accuracy_list_svm_10_mi + accuracy_list_xgb_10_mi + accuracy_list_lgb_10_mi +
                            accuracy_list_lr_20_mi + accuracy_list_rfc_20_mi + accuracy_list_svm_20_mi + accuracy_list_xgb_20_mi + accuracy_list_lgb_20_mi +
                            accuracy_list_lr_30_mi + accuracy_list_rfc_30_mi + accuracy_list_svm_30_mi + accuracy_list_xgb_30_mi + accuracy_list_lgb_30_mi +
                            accuracy_list_lr_50_mi + accuracy_list_rfc_50_mi + accuracy_list_svm_50_mi + accuracy_list_xgb_50_mi + accuracy_list_lgb_50_mi +
                            accuracy_list_lr_75_mi + accuracy_list_rfc_75_mi + accuracy_list_svm_75_mi + accuracy_list_xgb_75_mi + accuracy_list_lgb_75_mi +
                            accuracy_list_lr_90_mi + accuracy_list_rfc_90_mi + accuracy_list_svm_90_mi + accuracy_list_xgb_90_mi + accuracy_list_lgb_90_mi)

overall_f1_score_list_mi = (f1_score_list_lr_10_mi + f1_score_list_rfc_10_mi + f1_score_list_svm_10_mi + f1_score_list_xgb_10_mi + f1_score_list_lgb_10_mi +
                            f1_score_list_lr_20_mi + f1_score_list_rfc_20_mi + f1_score_list_svm_20_mi + f1_score_list_xgb_20_mi + f1_score_list_lgb_20_mi +
                            f1_score_list_lr_30_mi + f1_score_list_rfc_30_mi + f1_score_list_svm_30_mi + f1_score_list_xgb_30_mi + f1_score_list_lgb_30_mi +
                            f1_score_list_lr_50_mi + f1_score_list_rfc_50_mi + f1_score_list_svm_50_mi + f1_score_list_xgb_50_mi + f1_score_list_lgb_50_mi +
                            f1_score_list_lr_75_mi + f1_score_list_rfc_75_mi + f1_score_list_svm_75_mi + f1_score_list_xgb_75_mi + f1_score_list_lgb_75_mi +
                            f1_score_list_lr_90_mi + f1_score_list_rfc_90_mi + f1_score_list_svm_90_mi + f1_score_list_xgb_90_mi + f1_score_list_lgb_90_mi)

overall_auc_list_mi =  (auc_list_lr_10_mi + auc_list_rfc_10_mi + auc_list_svm_10_mi + auc_list_xgb_10_mi + auc_list_lgb_10_mi +
                        auc_list_lr_20_mi + auc_list_rfc_20_mi + auc_list_svm_20_mi + auc_list_xgb_20_mi + auc_list_lgb_20_mi +
                        auc_list_lr_30_mi + auc_list_rfc_30_mi + auc_list_svm_30_mi + auc_list_xgb_30_mi + auc_list_lgb_30_mi +
                        auc_list_lr_50_mi + auc_list_rfc_50_mi + auc_list_svm_50_mi + auc_list_xgb_50_mi + auc_list_lgb_50_mi +
                        auc_list_lr_75_mi + auc_list_rfc_75_mi + auc_list_svm_75_mi + auc_list_xgb_75_mi + auc_list_lgb_75_mi +
                        auc_list_lr_90_mi + auc_list_rfc_90_mi + auc_list_svm_90_mi + auc_list_xgb_90_mi + auc_list_lgb_90_mi)

overall_param_list_mi = (param_list_lr_10_mi + param_list_rfc_10_mi + param_list_svm_10_mi + param_list_xgb_10_mi + param_list_lgb_10_mi +
                            param_list_lr_20_mi + param_list_rfc_20_mi + param_list_svm_20_mi + param_list_xgb_20_mi + param_list_lgb_20_mi +
                            param_list_lr_30_mi + param_list_rfc_30_mi + param_list_svm_30_mi + param_list_xgb_30_mi + param_list_lgb_30_mi +
                            param_list_lr_50_mi + param_list_rfc_50_mi + param_list_svm_50_mi + param_list_xgb_50_mi + param_list_lgb_50_mi +
                            param_list_lr_75_mi + param_list_rfc_75_mi + param_list_svm_75_mi + param_list_xgb_75_mi + param_list_lgb_75_mi +
                            param_list_lr_90_mi + param_list_rfc_90_mi + param_list_svm_90_mi + param_list_xgb_90_mi + param_list_lgb_90_mi)

In [42]:
overall_accuracy_list_mrmr = (accuracy_list_lr_10_mrmr + accuracy_list_rfc_10_mrmr + accuracy_list_svm_10_mrmr + accuracy_list_xgb_10_mrmr + accuracy_list_lgb_10_mrmr +
                            accuracy_list_lr_20_mrmr + accuracy_list_rfc_20_mrmr + accuracy_list_svm_20_mrmr + accuracy_list_xgb_20_mrmr + accuracy_list_lgb_20_mrmr +
                            accuracy_list_lr_30_mrmr + accuracy_list_rfc_30_mrmr + accuracy_list_svm_30_mrmr + accuracy_list_xgb_30_mrmr + accuracy_list_lgb_30_mrmr +
                            accuracy_list_lr_50_mrmr + accuracy_list_rfc_50_mrmr + accuracy_list_svm_50_mrmr + accuracy_list_xgb_50_mrmr + accuracy_list_lgb_50_mrmr +
                            accuracy_list_lr_75_mrmr + accuracy_list_rfc_75_mrmr + accuracy_list_svm_75_mrmr + accuracy_list_xgb_75_mrmr + accuracy_list_lgb_75_mrmr +
                            accuracy_list_lr_90_mrmr + accuracy_list_rfc_90_mrmr + accuracy_list_svm_90_mrmr + accuracy_list_xgb_90_mrmr + accuracy_list_lgb_90_mrmr)

overall_f1_score_list_mrmr = (f1_score_list_lr_10_mrmr + f1_score_list_rfc_10_mrmr + f1_score_list_svm_10_mrmr + f1_score_list_xgb_10_mrmr + f1_score_list_lgb_10_mrmr +
                            f1_score_list_lr_20_mrmr + f1_score_list_rfc_20_mrmr + f1_score_list_svm_20_mrmr + f1_score_list_xgb_20_mrmr + f1_score_list_lgb_20_mrmr +
                            f1_score_list_lr_30_mrmr + f1_score_list_rfc_30_mrmr + f1_score_list_svm_30_mrmr + f1_score_list_xgb_30_mrmr + f1_score_list_lgb_30_mrmr +
                            f1_score_list_lr_50_mrmr + f1_score_list_rfc_50_mrmr + f1_score_list_svm_50_mrmr + f1_score_list_xgb_50_mrmr + f1_score_list_lgb_50_mrmr +
                            f1_score_list_lr_75_mrmr + f1_score_list_rfc_75_mrmr + f1_score_list_svm_75_mrmr + f1_score_list_xgb_75_mrmr + f1_score_list_lgb_75_mrmr +
                            f1_score_list_lr_90_mrmr + f1_score_list_rfc_90_mrmr + f1_score_list_svm_90_mrmr + f1_score_list_xgb_90_mrmr + f1_score_list_lgb_90_mrmr)

overall_auc_list_mrmr =  (auc_list_lr_10_mrmr + auc_list_rfc_10_mrmr + auc_list_svm_10_mrmr + auc_list_xgb_10_mrmr + auc_list_lgb_10_mrmr +
                        auc_list_lr_20_mrmr + auc_list_rfc_20_mrmr + auc_list_svm_20_mrmr + auc_list_xgb_20_mrmr + auc_list_lgb_20_mrmr +
                        auc_list_lr_30_mrmr + auc_list_rfc_30_mrmr + auc_list_svm_30_mrmr + auc_list_xgb_30_mrmr + auc_list_lgb_30_mrmr +
                        auc_list_lr_50_mrmr + auc_list_rfc_50_mrmr + auc_list_svm_50_mrmr + auc_list_xgb_50_mrmr + auc_list_lgb_50_mrmr +
                        auc_list_lr_75_mrmr + auc_list_rfc_75_mrmr + auc_list_svm_75_mrmr + auc_list_xgb_75_mrmr + auc_list_lgb_75_mrmr +
                        auc_list_lr_90_mrmr + auc_list_rfc_90_mrmr + auc_list_svm_90_mrmr + auc_list_xgb_90_mrmr + auc_list_lgb_90_mrmr)

overall_param_list_mrmr = (param_list_lr_10_mrmr + param_list_rfc_10_mrmr + param_list_svm_10_mrmr + param_list_xgb_10_mrmr + param_list_lgb_10_mrmr +
                            param_list_lr_20_mrmr + param_list_rfc_20_mrmr + param_list_svm_20_mrmr + param_list_xgb_20_mrmr + param_list_lgb_20_mrmr +
                            param_list_lr_30_mrmr + param_list_rfc_30_mrmr + param_list_svm_30_mrmr + param_list_xgb_30_mrmr + param_list_lgb_30_mrmr +
                            param_list_lr_50_mrmr + param_list_rfc_50_mrmr + param_list_svm_50_mrmr + param_list_xgb_50_mrmr + param_list_lgb_50_mrmr +
                            param_list_lr_75_mrmr + param_list_rfc_75_mrmr + param_list_svm_75_mrmr + param_list_xgb_75_mrmr + param_list_lgb_75_mrmr +
                            param_list_lr_90_mrmr + param_list_rfc_90_mrmr + param_list_svm_90_mrmr + param_list_xgb_90_mrmr + param_list_lgb_90_mrmr)

In [43]:
overall_accuracy_list_mi_mrmr = (accuracy_list_lr_10_mi_mrmr + accuracy_list_rfc_10_mi_mrmr + accuracy_list_svm_10_mi_mrmr + accuracy_list_xgb_10_mi_mrmr + accuracy_list_lgb_10_mi_mrmr +
                            accuracy_list_lr_20_mi_mrmr + accuracy_list_rfc_20_mi_mrmr + accuracy_list_svm_20_mi_mrmr + accuracy_list_xgb_20_mi_mrmr + accuracy_list_lgb_20_mi_mrmr +
                            accuracy_list_lr_30_mi_mrmr + accuracy_list_rfc_30_mi_mrmr + accuracy_list_svm_30_mi_mrmr + accuracy_list_xgb_30_mi_mrmr + accuracy_list_lgb_30_mi_mrmr +
                            accuracy_list_lr_50_mi_mrmr + accuracy_list_rfc_50_mi_mrmr + accuracy_list_svm_50_mi_mrmr + accuracy_list_xgb_50_mi_mrmr + accuracy_list_lgb_50_mi_mrmr +
                            accuracy_list_lr_75_mi_mrmr + accuracy_list_rfc_75_mi_mrmr + accuracy_list_svm_75_mi_mrmr + accuracy_list_xgb_75_mi_mrmr + accuracy_list_lgb_75_mi_mrmr +
                            accuracy_list_lr_90_mi_mrmr + accuracy_list_rfc_90_mi_mrmr + accuracy_list_svm_90_mi_mrmr + accuracy_list_xgb_90_mi_mrmr + accuracy_list_lgb_90_mi_mrmr)

overall_f1_score_list_mi_mrmr = (f1_score_list_lr_10_mi_mrmr + f1_score_list_rfc_10_mi_mrmr + f1_score_list_svm_10_mi_mrmr + f1_score_list_xgb_10_mi_mrmr + f1_score_list_lgb_10_mi_mrmr +
                            f1_score_list_lr_20_mi_mrmr + f1_score_list_rfc_20_mi_mrmr + f1_score_list_svm_20_mi_mrmr + f1_score_list_xgb_20_mi_mrmr + f1_score_list_lgb_20_mi_mrmr +
                            f1_score_list_lr_30_mi_mrmr + f1_score_list_rfc_30_mi_mrmr + f1_score_list_svm_30_mi_mrmr + f1_score_list_xgb_30_mi_mrmr + f1_score_list_lgb_30_mi_mrmr +
                            f1_score_list_lr_50_mi_mrmr + f1_score_list_rfc_50_mi_mrmr + f1_score_list_svm_50_mi_mrmr + f1_score_list_xgb_50_mi_mrmr + f1_score_list_lgb_50_mi_mrmr +
                            f1_score_list_lr_75_mi_mrmr + f1_score_list_rfc_75_mi_mrmr + f1_score_list_svm_75_mi_mrmr + f1_score_list_xgb_75_mi_mrmr + f1_score_list_lgb_75_mi_mrmr +
                            f1_score_list_lr_90_mi_mrmr + f1_score_list_rfc_90_mi_mrmr + f1_score_list_svm_90_mi_mrmr + f1_score_list_xgb_90_mi_mrmr + f1_score_list_lgb_90_mi_mrmr)

overall_auc_list_mi_mrmr =  (auc_list_lr_10_mi_mrmr + auc_list_rfc_10_mi_mrmr + auc_list_svm_10_mi_mrmr + auc_list_xgb_10_mi_mrmr + auc_list_lgb_10_mi_mrmr +
                        auc_list_lr_20_mi_mrmr + auc_list_rfc_20_mi_mrmr + auc_list_svm_20_mi_mrmr + auc_list_xgb_20_mi_mrmr + auc_list_lgb_20_mi_mrmr +
                        auc_list_lr_30_mi_mrmr + auc_list_rfc_30_mi_mrmr + auc_list_svm_30_mi_mrmr + auc_list_xgb_30_mi_mrmr + auc_list_lgb_30_mi_mrmr +
                        auc_list_lr_50_mi_mrmr + auc_list_rfc_50_mi_mrmr + auc_list_svm_50_mi_mrmr + auc_list_xgb_50_mi_mrmr + auc_list_lgb_50_mi_mrmr +
                        auc_list_lr_75_mi_mrmr + auc_list_rfc_75_mi_mrmr + auc_list_svm_75_mi_mrmr + auc_list_xgb_75_mi_mrmr + auc_list_lgb_75_mi_mrmr +
                        auc_list_lr_90_mi_mrmr + auc_list_rfc_90_mi_mrmr + auc_list_svm_90_mi_mrmr + auc_list_xgb_90_mi_mrmr + auc_list_lgb_90_mi_mrmr)

overall_param_list_mi_mrmr = (param_list_lr_10_mi_mrmr + param_list_rfc_10_mi_mrmr + param_list_svm_10_mi_mrmr + param_list_xgb_10_mi_mrmr + param_list_lgb_10_mi_mrmr +
                            param_list_lr_20_mi_mrmr + param_list_rfc_20_mi_mrmr + param_list_svm_20_mi_mrmr + param_list_xgb_20_mi_mrmr + param_list_lgb_20_mi_mrmr +
                            param_list_lr_30_mi_mrmr + param_list_rfc_30_mi_mrmr + param_list_svm_30_mi_mrmr + param_list_xgb_30_mi_mrmr + param_list_lgb_30_mi_mrmr +
                            param_list_lr_50_mi_mrmr + param_list_rfc_50_mi_mrmr + param_list_svm_50_mi_mrmr + param_list_xgb_50_mi_mrmr + param_list_lgb_50_mi_mrmr +
                            param_list_lr_75_mi_mrmr + param_list_rfc_75_mi_mrmr + param_list_svm_75_mi_mrmr + param_list_xgb_75_mi_mrmr + param_list_lgb_75_mi_mrmr +
                            param_list_lr_90_mi_mrmr + param_list_rfc_90_mi_mrmr + param_list_svm_90_mi_mrmr + param_list_xgb_90_mi_mrmr + param_list_lgb_90_mi_mrmr)

In [44]:
overall_accuracy_list_pca = (accuracy_list_lr_10_pca + accuracy_list_rfc_10_pca + accuracy_list_svm_10_pca + accuracy_list_xgb_10_pca + accuracy_list_lgb_10_pca +
                            accuracy_list_lr_20_pca + accuracy_list_rfc_20_pca + accuracy_list_svm_20_pca + accuracy_list_xgb_20_pca + accuracy_list_lgb_20_pca +
                            accuracy_list_lr_30_pca + accuracy_list_rfc_30_pca + accuracy_list_svm_30_pca + accuracy_list_xgb_30_pca + accuracy_list_lgb_30_pca +
                            accuracy_list_lr_50_pca + accuracy_list_rfc_50_pca + accuracy_list_svm_50_pca + accuracy_list_xgb_50_pca + accuracy_list_lgb_50_pca +
                            accuracy_list_lr_75_pca + accuracy_list_rfc_75_pca + accuracy_list_svm_75_pca + accuracy_list_xgb_75_pca + accuracy_list_lgb_75_pca +
                            accuracy_list_lr_90_pca + accuracy_list_rfc_90_pca + accuracy_list_svm_90_pca + accuracy_list_xgb_90_pca + accuracy_list_lgb_90_pca)

overall_f1_score_list_pca = (f1_score_list_lr_10_pca + f1_score_list_rfc_10_pca + f1_score_list_svm_10_pca + f1_score_list_xgb_10_pca + f1_score_list_lgb_10_pca +
                            f1_score_list_lr_20_pca + f1_score_list_rfc_20_pca + f1_score_list_svm_20_pca + f1_score_list_xgb_20_pca + f1_score_list_lgb_20_pca +
                            f1_score_list_lr_30_pca + f1_score_list_rfc_30_pca + f1_score_list_svm_30_pca + f1_score_list_xgb_30_pca + f1_score_list_lgb_30_pca +
                            f1_score_list_lr_50_pca + f1_score_list_rfc_50_pca + f1_score_list_svm_50_pca + f1_score_list_xgb_50_pca + f1_score_list_lgb_50_pca +
                            f1_score_list_lr_75_pca + f1_score_list_rfc_75_pca + f1_score_list_svm_75_pca + f1_score_list_xgb_75_pca + f1_score_list_lgb_75_pca +
                            f1_score_list_lr_90_pca + f1_score_list_rfc_90_pca + f1_score_list_svm_90_pca + f1_score_list_xgb_90_pca + f1_score_list_lgb_90_pca)

overall_auc_list_pca =  (auc_list_lr_10_pca + auc_list_rfc_10_pca + auc_list_svm_10_pca + auc_list_xgb_10_pca + auc_list_lgb_10_pca +
                        auc_list_lr_20_pca + auc_list_rfc_20_pca + auc_list_svm_20_pca + auc_list_xgb_20_pca + auc_list_lgb_20_pca +
                        auc_list_lr_30_pca + auc_list_rfc_30_pca + auc_list_svm_30_pca + auc_list_xgb_30_pca + auc_list_lgb_30_pca +
                        auc_list_lr_50_pca + auc_list_rfc_50_pca + auc_list_svm_50_pca + auc_list_xgb_50_pca + auc_list_lgb_50_pca +
                        auc_list_lr_75_pca + auc_list_rfc_75_pca + auc_list_svm_75_pca + auc_list_xgb_75_pca + auc_list_lgb_75_pca +
                        auc_list_lr_90_pca + auc_list_rfc_90_pca + auc_list_svm_90_pca + auc_list_xgb_90_pca + auc_list_lgb_90_pca)

overall_param_list_pca = (param_list_lr_10_pca + param_list_rfc_10_pca + param_list_svm_10_pca + param_list_xgb_10_pca + param_list_lgb_10_pca +
                            param_list_lr_20_pca + param_list_rfc_20_pca + param_list_svm_20_pca + param_list_xgb_20_pca + param_list_lgb_20_pca +
                            param_list_lr_30_pca + param_list_rfc_30_pca + param_list_svm_30_pca + param_list_xgb_30_pca + param_list_lgb_30_pca +
                            param_list_lr_50_pca + param_list_rfc_50_pca + param_list_svm_50_pca + param_list_xgb_50_pca + param_list_lgb_50_pca +
                            param_list_lr_75_pca + param_list_rfc_75_pca + param_list_svm_75_pca + param_list_xgb_75_pca + param_list_lgb_75_pca +
                            param_list_lr_90_pca + param_list_rfc_90_pca + param_list_svm_90_pca + param_list_xgb_90_pca + param_list_lgb_90_pca)

In [45]:
print(len(overall_accuracy_list_mi))
print(len(overall_f1_score_list_mi))
print(len(overall_auc_list_mi))
print(len(overall_param_list_mi))

print(len(overall_accuracy_list_mrmr))
print(len(overall_f1_score_list_mrmr))
print(len(overall_auc_list_mrmr))
print(len(overall_param_list_mrmr))

print(len(overall_accuracy_list_mi_mrmr))
print(len(overall_f1_score_list_mi_mrmr))
print(len(overall_auc_list_mi_mrmr))
print(len(overall_param_list_mi_mrmr))

print(len(overall_accuracy_list_pca))
print(len(overall_f1_score_list_pca))
print(len(overall_auc_list_pca))
print(len(overall_param_list_pca))

300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300


In [46]:
results_dictionary = {
    'samples': filename_sample_list_final,
    'models': models,
    'percentiles': percentiles,
    'mi_accuracy': overall_accuracy_list_mi,
    'mi_f1_score': overall_f1_score_list_mi,
    'mi_auc': overall_auc_list_mi,
    'mrmr_accuracy': overall_accuracy_list_mrmr,
    'mrmr_f1_score': overall_f1_score_list_mrmr,
    'mrmr_auc': overall_auc_list_mrmr,
    'mi_mrmr_accuracy': overall_accuracy_list_mi_mrmr,
    'mi_mrmr_f1_score': overall_f1_score_list_mi_mrmr,
    'mi_mrmr_auc': overall_auc_list_mi_mrmr,
    'pca_accuracy': overall_accuracy_list_pca,
    'pca_f1_score': overall_f1_score_list_pca,
    'pca_auc': overall_auc_list_pca,
    'mi_params': overall_param_list_mi,
    'mrmr_params': overall_param_list_mrmr,
    'mi_mrmr_params': overall_param_list_mi_mrmr,
    'pca_params': overall_param_list_pca
}
results_df = pd.DataFrame(results_dictionary)

results_df.to_csv('/Users/nitanshjain/Documents/Projects/Shopper_Intent_Prediction/shopper-intent-prediction/long_trajectory/results/overall_results_20_v2.csv', index=False)